## Authentication

In [1]:
! python HelloAnalytics.py

#Si la autentificación se hace con éxito devuelve el número total de sesiones de los últimos 7 días:

Date range (0)
ga:sessions: 11103


In [ ]:
# me dio problema al autentificar
# si vuelve a pasar ir a https://console.developers.google.com/flows/enableapi?apiid=analyticsreporting.googleapis.com&credential=client_key&hl=es
# seleccionar proyecto DSproject2021
# ir a APIS y SERVICIOS > Panel de control > Credenciales
# No sé si fue xq tenia que informarme de las nuevas condiciones de servicio > no tuve q aceptar solo ir a ellas en una mensaje q me salio
# o que entre en quikstart en credenciales aunq no hice nada
# la cuestion es que al volver a ejecutar el comando ! python Hello Analytics funcionó y me redirecciono para volver a autorizar a la app para darle acceso a mis datos de analytics
# continue y acepté todo y la autenticacion se hizo con exito

## First API Call for testing

In [3]:
# Revisar el contenido del fichero HelloAnalytics.py para ver la estructura que sigue para realizar el request:

! cat HelloAnalytics.py

"""Hello Analytics Reporting API V4."""

import argparse

from apiclient.discovery import build
import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
CLIENT_SECRETS_PATH = 'client_secrets.json' # Path to client_secrets.json file.
VIEW_ID = '151304036'


def initialize_analyticsreporting():
  """Initializes the analyticsreporting service object.

  Returns:
    analytics an authorized analyticsreporting service object.
  """
  # Parse command-line arguments.
  parser = argparse.ArgumentParser(
      formatter_class=argparse.RawDescriptionHelpFormatter,
      parents=[tools.argparser])
  flags = parser.parse_args([])

  # Set up a Flow object to be used if we need to authenticate.
  flow = client.flow_from_clientsecrets(
      CLIENT_SECRETS_PATH, scope=SCOPES,
      message=tools.message_if_missing(CLIENT_SECRETS_PATH))

  # Prepare credentials, and authorize HTTP obje

In [7]:
#Importar librerías

import argparse

from apiclient.discovery import build
import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools

In [10]:
# Definir la función para crear las credenciales de servicio:

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
CLIENT_SECRETS_PATH = 'client_secrets.json' # Path to client_secrets.json file.
VIEW_ID = '151304036'

def initialize_analyticsreporting():
  """Initializes the analyticsreporting service object.

  Returns:
    analytics an authorized analyticsreporting service object.
  """
  # Parse command-line arguments.
  parser = argparse.ArgumentParser(
      formatter_class=argparse.RawDescriptionHelpFormatter,
      parents=[tools.argparser])
  flags = parser.parse_args([])

  # Set up a Flow object to be used if we need to authenticate.
  flow = client.flow_from_clientsecrets(
      CLIENT_SECRETS_PATH, scope=SCOPES,
      message=tools.message_if_missing(CLIENT_SECRETS_PATH))

  # Prepare credentials, and authorize HTTP object with them.
  # If the credentials don't exist or are invalid run through the native client
  # flow. The Storage object will ensure that if successful the good
  # credentials will get written back to a file.
  storage = file.Storage('analyticsreporting.dat')
  credentials = storage.get()
  if credentials is None or credentials.invalid:
    credentials = tools.run_flow(flow, storage, flags)
  http = credentials.authorize(http=httplib2.Http())

  # Build the service object.
  analytics = build('analyticsreporting', 'v4', http=http)

  return analytics

In [11]:
# Crear a service object:

service = initialize_analyticsreporting()

In [18]:
# Report Request sencillo de prueba
# una dimensión, una métrica

response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'30daysAgo','endDate':'today'}],
        'metrics':[{'expression': 'ga:Sessions'}],
        'dimensions':[{'name':'ga:pagePath'}],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        'pageSize':100
    }]
}).execute()

In [19]:
# Parsear el report:

dim = []
val = []

for report in response.get('reports',[]):
    
    columnHeader = report.get('columnHeader',{})
    dimensionHeaders = columnHeader.get('dimensions',[])
    metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])
    
    rows = report.get('data',{}).get('rows',[])
    for row in rows:
        dimensions = row.get('dimensions',[])
        dateRangeValues=row.get('metrics',[])
        
        for header, dimension in zip(dimensionHeaders,dimensions):
            dim.append(dimension)
        
        for i,values in enumerate (dateRangeValues):
            for metricHeader,value in zip(metricHeaders,values.get('values')):
                val.append(int(value))

In [20]:
# Crear un df con los datos

import pandas as pd

val.reverse()
dim.reverse()

df=pd.DataFrame()
df['Sessions']=val
df['pagePath']=dim
df = df[["pagePath","Sessions"]]

In [21]:
df.head()

,pagePath,Sessions
0,/es/productos-gema-herrerias/6446-emulsion-jap...,79
1,/es/serum/6927-svr-sebiaclear-serum-.html,80
2,/es/exfoliantes/4851-cristina-mitre-gh5azelaic...,80
3,/es/serum/5129-farma-dorsch-suerum-sérum-de-hi...,81
4,/es/serum/1307-farma-dorsch-go-organic-serum-b...,81


In [ ]:
# Prueba con varias dimensiones y metricas

In [174]:
response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'7daysAgo','endDate':'today'}],
        'metrics':
        [
            {'expression':'ga:sessions'},
            {'expression':'ga:users'},
        ],
        'dimensions':
        [
            {'name':'ga:pagePath'},
            {'name':'ga:sourceMedium'},
            
            
        ],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        #'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        'pageSize':100000
    }]
}).execute()

In [175]:
# Parsear el report:

list_ = []

for report in response.get('reports',[]):
    
    columnHeader = report.get('columnHeader',{})
    dimensionHeaders = columnHeader.get('dimensions',[])
    metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])
    
    rows = report.get('data',{}).get('rows',[])
    for row in rows:
        dict_={}
        dimensions = row.get('dimensions',[])
        dateRangeValues=row.get('metrics',[])
        
        for header, dimension in zip(dimensionHeaders,dimensions):
            dict_[header]=dimension
        
        for i,values in enumerate (dateRangeValues):
            for metricHeader,value in zip(metricHeaders,values.get('values')):
                if ',' in value or '.' in value:
                  dict_[metricHeader.get('name')] = float(value)
                else:
                  dict_[metricHeader.get('name')] = int(value)
                
        list_.append(dict_)

In [176]:
# crear df con los datos:

df = pd.DataFrame(list_)
df.shape

(4440, 4)

In [177]:
df.head()

,ga:pagePath,ga:sourceMedium,ga:sessions,ga:users
0,(not set),(direct) / (none),74,74
1,(not set),google / cpc,1,1
2,/adminFarmac1a$/index.php?controller=AdminDash...,(direct) / (none),3,3
3,/adminFarmac1a$/index.php?controller=AdminDash...,google / organic,1,1
4,/es/,(direct) / (none),386,357


## Data Collection for my analysis

El objetivo de este proyecto es identificar posibles factores que permitan mejorar la tasa de conversion del ecommerce en cuestión. Dicho ratio es el resultado de dividir el número de conversiones por el número de visitas a un sitio web. 

En este caso, centraremos nuestros esfuerzos en mejorar el numerador de nuestra fracción. 

Buscamos predecir la conversión del usuario en función del máximo de dimensiones que pueden influir en él y que nosotros podamos capturar. 

### Nivel de agregación de los datos

El objetivo de este proyecto es identificar posibles factores que permitan mejorar la tasa de conversion del ecommerce en cuestión. Dicho ratio es el resultado de dividir el número de conversiones por el número de visitas a un sitio web. 

En este caso, centraremos nuestros esfuerzos en mejorar el numerador de nuestra fracción. 

Buscamos predecir la conversión del usuario en función del máximo de dimensiones que pueden influir en él y que nosotros podamos capturar. 

Dado que solo la versión premium de Google Analytics (precio mínimo: 150.000 euros/año) da acceso a todos los hits de una sesión de un usuario, he reformulado el planteamiento para estudiar los features que determinan que un determinado producto sea o no añadido al carrito.

Se ha escogido esta variable target porque, aunque la conversión más fiable y definitiva sea la compra, ésta se produce en la misma pasarela de pago para todos los productos y perderíamos información sobre las características de la página que motivaron el evento de conversión. 

Por tanto, nuestra variable target será la metríca productAddsToCart que solicitaremos para cada uno de los productos de la web. Ahora bien, al hacer esta consulta, la API de Google Analytics nos devuelve un informe agregado para el periodo solicitado (en este caso los últimos 7 días) con el sumatorio de veces que se ha añadido el producto: 

In [192]:
#REPORT REQUEST:
response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'7daysAgo','endDate':'today'}],
        'metrics':
        [
            {'expression':'ga:productAddsToCart'},
        ],
        'dimensions':
        [
            {'name':'ga:productName'},    
        ],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        #'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        'pageSize':100000
    }]
}).execute()


#PARSING THE REPORT DATA:
list_ = []

for report in response.get('reports',[]):
    
    columnHeader = report.get('columnHeader',{})
    dimensionHeaders = columnHeader.get('dimensions',[])
    metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])
    
    rows = report.get('data',{}).get('rows',[])
    for row in rows:
        dict_={}
        dimensions = row.get('dimensions',[])
        dateRangeValues=row.get('metrics',[])
        
        for header, dimension in zip(dimensionHeaders,dimensions):
            dict_[header]=dimension
        
        for i,values in enumerate (dateRangeValues):
            for metricHeader,value in zip(metricHeaders,values.get('values')):
                if ',' in value or '.' in value:
                  dict_[metricHeader.get('name')] = float(value)
                else:
                  dict_[metricHeader.get('name')] = int(value)
                
        list_.append(dict_)
        
#DATAFRAME WITH DATA:
df = pd.DataFrame(list_)
print(df.shape)
df.head()

(502, 2)


,ga:productName,ga:productAddsToCart
0,▷ ▷HELIOCARE 360 Water Gel SPF 50+,6
1,▷▷ CREMA REAFIRMANTE SENO-CORPO 250ml - GUAM,1
2,▷▷ Envío Gratuito|Skinceuticals Pack Resveratr...,6
3,▷▷ GUAM - FANGO GEL FIR ACCION 300 ml ANTICEL...,2
4,▷▷Emulsión limpiadora JAPONESA Kire Gema Herre...,13


Con el fin de desagregar estos valores, hasta alcanzar un scope usuario/sesión, añadiré la variable temporal dateHourMinute y otras dimensiones específicas de usuario, que también nos resultarán de utilidad más tarde para el analisis de los factores. 

> PONER LA DESCRIPCION DE LAS VARIABLES QUE VAMOS A INCORPORAR 

In [194]:
#REPORT REQUEST:
response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'7daysAgo','endDate':'today'}],
        'metrics':
        [
            {'expression':'ga:productAddsToCart'},
        ],
        'dimensions':
        [
            {'name':'ga:productName'},
            {'name':'ga:pagePath'},
            {'name':'ga:dateHourMinute'},
            {'name':'ga:sessionCount'},
            {'name':'ga:daysSinceLastSession'},
            
            
        ],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        #'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        'pageSize':100000
    }]
}).execute()


#PARSING THE REPORT DATA:
list_ = []

for report in response.get('reports',[]):
    
    columnHeader = report.get('columnHeader',{})
    dimensionHeaders = columnHeader.get('dimensions',[])
    metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])
    
    rows = report.get('data',{}).get('rows',[])
    for row in rows:
        dict_={}
        dimensions = row.get('dimensions',[])
        dateRangeValues=row.get('metrics',[])
        
        for header, dimension in zip(dimensionHeaders,dimensions):
            dict_[header]=dimension
        
        for i,values in enumerate (dateRangeValues):
            for metricHeader,value in zip(metricHeaders,values.get('values')):
                if ',' in value or '.' in value:
                  dict_[metricHeader.get('name')] = float(value)
                else:
                  dict_[metricHeader.get('name')] = int(value)
                
        list_.append(dict_)
        
#DATAFRAME WITH DATA:
df = pd.DataFrame(list_)
print(df.shape)
df.head()

(2113, 6)


,ga:productName,ga:pagePath,ga:dateHourMinute,ga:sessionCount,ga:daysSinceLastSession,ga:productAddsToCart
0,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/protectores-solares/4402-heliocare-360-wat...,202101311412,1,0,1
1,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/protectores-solares/4402-heliocare-360-wat...,202101311445,1,0,1
2,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/protectores-solares/4402-heliocare-360-wat...,202101311520,1,0,2
3,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/protectores-solares/4402-heliocare-360-wat...,202101311842,1,0,1
4,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/protectores-solares/4402-heliocare-360-wat...,202101311944,1,0,1


In [195]:
df[df['ga:productAddsToCart']>1].shape

(81, 6)

Nuestro nivel de agregación se ha reducido drasticamente para la columna ga:productAddsToCart y ahora en principio recoge hits.  No obstante, se observa que todavía algunas observaciones (81 de 2113) tienen un valor mayor que 1 para esta columna.

GA describe esta métrica como el Number of times the product was added to the shopping cart. En principio deberia ser una metrica que contara las veces que se pincha sobre el botón 'añadir a carrito' para un producto determinado. En este caso, tenemos ese valor para un día y una hora determinada, así como para otras dimensiones que deberían conducirnos a describir a un usuario unico sessionCount (session Index for a user) y daysSinceLastSession (The number of days elapsed since users last visited the proper).

No parece probable que haya un mismo usuario con las mismas caracteristicas, el mismo día a la misma hora, añadiendo el mismo producto al carrito. Parece más razonable pensar que se trata del mismo usuario que ha hecho click varias veces en el botón 'añadir al carrito'. 

En cualquier caso, exploremos un poco más. Acotaremos aún más el analisis añadiendo la dimensión Source/Medium que indica la procedencia del usuario y nos aportará mayor nivel de segmentación. 

In [203]:
#REPORT REQUEST:
response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'7daysAgo','endDate':'today'}],
        'metrics':
        [
            {'expression':'ga:productAddsToCart'},
        ],
        'dimensions':
        [
            {'name':'ga:productName'},
            {'name':'ga:pagePath'},
            {'name':'ga:dateHourMinute'},
            {'name':'ga:sessionCount'},
            {'name':'ga:daysSinceLastSession'},
            {'name':'ga:sourceMedium'},
            
        ],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        #'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        'pageSize':100000
    }]
}).execute()


#PARSING THE REPORT DATA:
list_ = []

for report in response.get('reports',[]):
    
    columnHeader = report.get('columnHeader',{})
    dimensionHeaders = columnHeader.get('dimensions',[])
    metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])
    
    rows = report.get('data',{}).get('rows',[])
    for row in rows:
        dict_={}
        dimensions = row.get('dimensions',[])
        dateRangeValues=row.get('metrics',[])
        
        for header, dimension in zip(dimensionHeaders,dimensions):
            dict_[header]=dimension
        
        for i,values in enumerate (dateRangeValues):
            for metricHeader,value in zip(metricHeaders,values.get('values')):
                if ',' in value or '.' in value:
                  dict_[metricHeader.get('name')] = float(value)
                else:
                  dict_[metricHeader.get('name')] = int(value)
                
        list_.append(dict_)
        
#DATAFRAME WITH DATA:
df = pd.DataFrame(list_)
print(df.shape)
df.head()

(2113, 7)


,ga:productName,ga:pagePath,ga:dateHourMinute,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productAddsToCart
0,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/protectores-solares/4402-heliocare-360-wat...,202101311412,1,0,(direct) / (none),1
1,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/protectores-solares/4402-heliocare-360-wat...,202101311445,1,0,(direct) / (none),1
2,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/protectores-solares/4402-heliocare-360-wat...,202101311520,1,0,(direct) / (none),2
3,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/protectores-solares/4402-heliocare-360-wat...,202101311842,1,0,(direct) / (none),1
4,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/protectores-solares/4402-heliocare-360-wat...,202101311944,1,0,google / organic,1


In [204]:
df[df['ga:productAddsToCart']>1].shape

(81, 7)

A pesar de haber añadido una nueva variable descriptiva del usuario, seguimos teniendo el mismo número de observaciones (2113) y de filas con ga:productAddsToCart mayor que 1 (81). Veamos ahora su detalle:

In [208]:
addsToCart_higher_one = df[df['ga:productAddsToCart']>1]
addsToCart_higher_one.sample(50)

,ga:productName,ga:pagePath,ga:dateHourMinute,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productAddsToCart
1575,Retincare 30 ML,/es/top-ventas/1026-retincare-comprar-oferta-m...,202102011612,1,0,google / cpc,2
581,Farma Dorsch Leche corporal ultrareafirmante 4...,/es/cuidado-corporal/1004-farma-dorsch-leche-c...,202102021252,1,0,google / cpc,2
472,Envío Gratuito |MEDIK8 r-Retinoate Intense,/es/inicio/4570-medik8-r-retinoate-81862502143...,202101291223,1,0,google / organic,6
1584,Retincare 30 ML,/es/top-ventas/1026-retincare-comprar-oferta-m...,202102021349,1,0,google / cpc,3
120,Antiage Hair Boost Fridda Dorsch,/es/productos-capilares/4599-antiage-hair-boos...,202101291857,1,0,google / cpc,2
325,Champú Go Organic PELO GRASO Anticaida Farma D...,/es/top-ventas/1306-farma-dorsch-go-organic-ch...,202101301721,1,0,(direct) / (none),2
692,Gema Herrerias AZA-GZ crema multifunción | El ...,/es/productos-gema-herrerias/7574-gh-aza-gz-cr...,202101311421,1,0,google / cpc,5
1076,Mascarilla NEGRA Adulto Respiratoria FFP2 LOV ...,/es/productos-covid/7840--mascarilla-negra-adu...,202101280847,1,0,instagram.com / referral,2
474,Envío Gratuito|Skinceuticals Age EYE Complex C...,/es/carrito,202101291910,1,0,google / organic,7
122,Antiage Hair Boost Fridda Dorsch,/es/productos-capilares/4599-antiage-hair-boos...,202101291858,2,0,google / organic,2


In [211]:
addsToCart_higher_one['ga:sourceMedium'].value_counts()

google / organic            37
google / cpc                28
(direct) / (none)           10
instagram.com / referral     2
yahoo / organic              1
sendinblue / email           1
IGShopping / Social          1
bing / organic               1
Name: ga:sourceMedium, dtype: int64

En principio, la mayor parte de las observaciones proceden de Motores de Busqueda y como comentaba anteriormente parece improbable que varios usuarios con caracteristicas similares coincidan en buscar el mismo producto y hagan click en el boton de añadir a carrito del producto el mismo día a la misma hora. 

Solo parece probable en el caso del lanzamiento de una campaña que haya sido vista al mismo tiempo por varios usuarios pero no parece el caso.

Por tanto, de cara a nuestro analisis, consideraremos que se trata del mismo usuario que ha pinchado varias veces sobre el botón.  

### No Add_to_Cart Event

Hasta ahora, hemos considerado unicamente aquellas observaciones en las que se añade un producto al carrito. Sin embargo, para nuestro analísis, necesitamos considerar también aquellos escenarios en los que el usuario tuvo la oportunidad de añadir el producto al carrito pero no lo hizo.

Para ello, incluiremos las variables productListViews (si el usuario vio el producto en una página tipo lista. GA lo contabiliza como impresiones. En realidad el usuario no tiene porq haberse fijado o interactuado con el), productListClicks (Number of times users clicked the product when it appeared in the product list) y productDetailViews (si el usuario vio el producto en su página especifica)

> especificar definiciones de variables y aclarar cuales son las listviews y las detailviews
> There are three main types of product lists:

Categories – Categories are the most basic way that products are grouped. By tracking category lists, you can see how products perform in the section of the site where they get the most exposure.
Search Results – Search results are lists of products created by a particular user’s search query. While there are infinite search query possibilities, popular search queries are worth optimizing to increase conversion rates.
Related Products – Related products lists often appear on a product detail pages and consist of other products that customers tend to purchase together. By analyzing this data, you can identify the best cross-selling opportunities.

In [215]:
#REPORT REQUEST:
response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'7daysAgo','endDate':'today'}],
        'metrics':
        [
            {'expression':'ga:productListViews'},
            {'expression':'ga:productListClicks'},
            {'expression':'ga:productDetailViews'},
            {'expression':'ga:productAddsToCart'},
        ],
        'dimensions':
        [
            {'name':'ga:productName'},
            {'name':'ga:pagePath'},
            {'name':'ga:dateHourMinute'},
            {'name':'ga:sessionCount'},
            {'name':'ga:daysSinceLastSession'},
            {'name':'ga:sourceMedium'},
            
        ],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        #'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        'pageSize':100000
    }]
}).execute()


#PARSING THE REPORT DATA:
list_ = []

for report in response.get('reports',[]):
    
    columnHeader = report.get('columnHeader',{})
    dimensionHeaders = columnHeader.get('dimensions',[])
    metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])
    
    rows = report.get('data',{}).get('rows',[])
    for row in rows:
        dict_={}
        dimensions = row.get('dimensions',[])
        dateRangeValues=row.get('metrics',[])
        
        for header, dimension in zip(dimensionHeaders,dimensions):
            dict_[header]=dimension
        
        for i,values in enumerate (dateRangeValues):
            for metricHeader,value in zip(metricHeaders,values.get('values')):
                if ',' in value or '.' in value:
                  dict_[metricHeader.get('name')] = float(value)
                else:
                  dict_[metricHeader.get('name')] = int(value)
                
        list_.append(dict_)
        
#DATAFRAME WITH DATA:
df = pd.DataFrame(list_)
print(df.shape)
df.head()

(100000, 10)


,ga:productName,ga:pagePath,ga:dateHourMinute,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart
0,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1041-neostrata-basis-redox-crema-antiarrug...,202101301913,1,0,google / organic,1,0,0,0
1,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1041-neostrata-basis-redox-crema-antiarrug...,202101310813,3,0,google / organic,1,0,0,0
2,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1041-neostrata-basis-redox-crema-antiarrug...,202101311714,2,0,(direct) / (none),1,0,0,0
3,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1041-neostrata-basis-redox-crema-antiarrug...,202101312017,5,0,google / organic,1,0,0,0
4,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1041-neostrata-basis-redox-crema-antiarrug...,202101312034,5,0,google / organic,1,0,0,0


In [221]:
df['hits_combinations'] = df["ga:productListViews"].apply(lambda x: str(x))+ "." + df["ga:productListClicks"].apply(lambda x: str(x)) + "." + df["ga:productDetailViews"].apply(lambda x: str(x)) + "." + df["ga:productAddsToCart"].apply(lambda x: str(x)) 

In [222]:
df.head()

,ga:productName,ga:pagePath,ga:dateHourMinute,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart,hits_combinations
0,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1041-neostrata-basis-redox-crema-antiarrug...,202101301913,1,0,google / organic,1,0,0,0,1.0.0.0
1,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1041-neostrata-basis-redox-crema-antiarrug...,202101310813,3,0,google / organic,1,0,0,0,1.0.0.0
2,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1041-neostrata-basis-redox-crema-antiarrug...,202101311714,2,0,(direct) / (none),1,0,0,0,1.0.0.0
3,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1041-neostrata-basis-redox-crema-antiarrug...,202101312017,5,0,google / organic,1,0,0,0,1.0.0.0
4,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1041-neostrata-basis-redox-crema-antiarrug...,202101312034,5,0,google / organic,1,0,0,0,1.0.0.0


In [223]:
df['hits_combinations'].value_counts()

1.0.0.0    88563
0.0.1.0     5145
2.0.0.0     2562
0.0.2.0     1292
1.1.0.0      561
0.0.3.0      386
0.0.1.1      376
0.1.0.0      250
0.0.0.1      249
3.0.0.0      167
2.1.0.0      131
0.0.2.1       87
1.0.0.1       68
4.0.0.0       31
0.0.4.0       22
3.1.0.0       17
0.0.3.1       15
1.0.1.0       12
0.0.5.0        9
0.0.1.2        7
0.0.0.2        6
0.0.0.3        4
0.0.1.3        3
0.0.2.2        3
1.2.0.0        3
2.0.0.1        3
0.0.4.1        2
0.2.0.0        2
3.2.0.0        2
2.2.0.0        2
1.0.2.0        2
0.7.0.0        1
2.1.0.1        1
0.2.1.0        1
1.0.0.2        1
3.0.0.1        1
0.0.2.6        1
1.0.3.0        1
0.0.0.5        1
1.3.0.0        1
0.0.3.2        1
0.0.6.0        1
4.1.0.0        1
0.0.2.4        1
0.0.4.2        1
0.0.0.4        1
0.0.0.7        1
1.1.1.0        1
0.0.5.1        1
Name: hits_combinations, dtype: int64

In [224]:
def combi_transf(x):
    if x>1:
        x=str(1)
    else:
        x=str(x)
    return x

In [225]:
df['hits_combinations'] = df["ga:productListViews"].apply(combi_transf)+ "." + df["ga:productListClicks"].apply(combi_transf) + "." + df["ga:productDetailViews"].apply(combi_transf) + "." + df["ga:productAddsToCart"].apply(combi_transf)

In [227]:
df['hits_combinations'].value_counts()

1.0.0.0    91323
0.0.1.0     6855
1.1.0.0      718
0.0.1.1      498
0.0.0.1      262
0.1.0.0      253
1.0.0.1       73
1.0.1.0       15
1.1.0.1        1
1.1.1.0        1
0.1.1.0        1
Name: hits_combinations, dtype: int64

In [230]:
#observaciones 1.0.0.0
# se muestra producto al usuario en una lista pero este no interactúa de ningún modo con él

obs_1000 = df[df['hits_combinations']=="1.0.0.0"]
obs_1000.sample(50)

# en general, este tipo de observaciones tienen lugar en páginas referidas a categorías o resultados de busqueda interna.
# aunque también encontramos urls de fichas de producto. 
# En este último caso considera lista a la seccion "Los clientes que adquirieron este producto también compraron" al final de la pagina.

,ga:productName,ga:pagePath,ga:dateHourMinute,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart,hits_combinations
31441,"CALMIOX RECTAL 1 MG/G CREMA RECTAL , 1 TUBO DE...",/es/contorno-de-ojos-para-bolsas/6339-skinperf...,202102011902,1,0,google / organic,1,0,0,0,1.0.0.0
61192,Esthederm Intensivo Hialurónico Sérum 30ml,/es/828-serum,202102022224,1,0,(direct) / (none),1,0,0,0,1.0.0.0
33099,Caudalie Espuma Limpiadora VINOCLEAN 150ml,/es/,202101300941,8,3,(direct) / (none),1,0,0,0,1.0.0.0
62581,Esthederm Intensivo Retinol Crema 50ml,/es/,202102031948,19,0,google / organic,1,0,0,0,1.0.0.0
27199,"Blanc Supplements BE HEALTHY, BE BEAUTIFUL- Sk...",/es/brand/608-blanc-supplements,202102011320,3,0,google / cpc,1,0,0,0,1.0.0.0
59032,Esthederm Agua Celular Crema 50ml,/es/880-hidratantes-premium?page=2,202102031845,1,0,google / organic,1,0,0,0,1.0.0.0
94954,GYNELAUDE MUCUS GEL VAGINAL 30 ML,/es/top-ventas/6262-skinceuticals-retinol-03-3...,202101301200,2,7,sendinblue / email,1,0,0,0,1.0.0.0
70746,Farma Dorsch Go Organic Aloe Vera Gel 200 ml,/es/903-top-ventas?page=2,202101281515,1,0,google / organic,1,0,0,0,1.0.0.0
35331,CAUDALIE RESVERATROL SERUM FIRMEZA 30ML,/es/826-hidratantes,202101292046,2,0,(direct) / (none),1,0,0,0,1.0.0.0
43129,Crema Funcion Barrera EDICION LIMITADA Gema He...,/es/brand/356-gema-herrerias,202101291221,1,0,google / cpc,1,0,0,0,1.0.0.0


In [233]:
## observaciones 0.0.1.0 en las que el usuario ve el producto en su url especifica pero no lo añade al carrito

obs_0010 = df[df['hits_combinations']=="0.0.1.0"]
obs_0010.sample(50)

# en efecto, parece que todos las observaciones tienen lugar en una ficha de producto especifica.

,ga:productName,ga:pagePath,ga:dateHourMinute,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart,hits_combinations
487,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/protectores-solares/4402-heliocare-360-wat...,202101311529,1,0,(direct) / (none),0,0,5,0,0.0.1.0
44234,CumLaude Hydra Spray Bruma Hidratante Vulvar 75ml,/es/hidratantes/7828-cumlaude-hydra-spray-brum...,202102021118,1,0,google / organic,0,0,1,0,0.0.1.0
17140,Aquilea Pack Articulaciones Colágeno + Magnesi...,/es/vitaminas/7563-aquilea-colageno-magnesio-4...,202102021728,1,0,google / organic,0,0,1,0,0.0.1.0
36286,Caudalie VINOCLEAN Locion Tonica Hidratante 100ml,/es/tonicos/7214-caudalie-vinoclean-locion-ton...,202101290003,4,15,l.instagram.com / referral,0,0,1,0,0.0.1.0
87966,Gema Herrerias Crema Función Barrera Diaria 50 ml,/es/productos-gema-herrerias/6751-gema-herreri...,202102041110,1,0,google / organic,0,0,1,0,0.0.1.0
95968,HELIOCARE 360 CUSHION COMPACT BRONZE INTENSE S...,/es/protectores-solares/6361-friendly-url-auto...,202102030116,5,0,google / organic,0,0,1,0,0.0.1.0
70439,Farma Dorsch Crema Antiarrugas Intensiva Vitam...,/es/top-ventas/1318-farma-dorsch-vit-ace-compr...,202101301648,1,0,google / cpc,0,0,1,0,0.0.1.0
18397,Bakuchiol Peptidos MEDIK8 Alternativa al Retin...,/es/inicio/6900-bakuchiol-peptides-medik8-alte...,202101291728,2,0,google / cpc,0,0,1,0,0.0.1.0
27079,Blanc Supplements B-Brilliant- Rhodiola + Vit ...,/es/vitaminas-minerales-y-suplementos/7001-bla...,202102022337,1,0,google / organic,0,0,1,0,0.0.1.0
50885,Elixir Colágeno Marino de Goah Clínic 20 viales,/es/vitaminas-minerales-y-suplementos/6321-eli...,202101280104,1,0,google / organic,0,0,1,0,0.0.1.0


In [234]:
# observaciones 1.1.0.0
# El producto aparece en una lista y el usuario hace click sobre él para acceder a la ficha de producto

obs_1100 = df[df['hits_combinations']=="1.1.0.0"]
obs_1100.sample(50)

# como en el caso de 1.0.0.0
# gran parte de las observaciones tienen lugar en una página correspondiente a una categoria de productos o a una busqueda interna.
# Sin embargo, también tenemos muchas observaciones en las que el usuario ha hecho click sobre el producto en la lista de recomendados en el interior de la ficha de otra producto

,ga:productName,ga:pagePath,ga:dateHourMinute,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart,hits_combinations
54035,Envío Gratuito | MEDIK8 Refining Moisturiser 50ml,/es/brand/578-medik8,202102031848,8,0,(direct) / (none),1,1,0,0,1.1.0.0
35048,CAUDALIE RESVERATROL CREMA TISANA DE NOCHE 50ML,/es/inicio/4149-caudalie-resveratrol-lift-crem...,202101301319,1,0,google / organic,1,1,0,0,1.1.0.0
18867,Bálsamo de Ozono Ozoaqua TOP 5 TELVA,/es/903-top-ventas?page=2,202101290925,1,0,google / organic,2,1,0,0,1.1.0.0
92879,Glicoisdin 8% Crema facial antiedad con ácido ...,/es/brand/559-isdin?page=2,202101291324,1,0,google / organic,2,1,0,0,1.1.0.0
69378,Farma Dorsch Contorno de Ojos + EGF,/es/contorno-de-ojos-para-ojera-oscura/1313-fa...,202102032047,1,0,google / cpc,1,2,0,0,1.1.0.0
53870,ENVIO GRATIS | Farma Dorsch Serum Antipolucion...,/es/brand/331-fridda-dorsch-farma-dorsch,202101291817,2,1,google / organic,1,1,0,0,1.1.0.0
49406,Dr. Arthouros Alba Serum Despigmentante Amplio...,/es/module/iqitsearch/searchiqit,202101290028,5,0,google / cpc,1,1,0,0,1.1.0.0
69498,Farma Dorsch Contorno Ojos Your Eyes de Fridda...,/es/1015-contorno-de-ojos-para-ojera-oscura,202101291400,1,0,google / organic,1,1,0,0,1.1.0.0
27853,Blanc Supplements D-PURE- Aloe Complex 30 cáps...,/es/914-vitaminas-minerales-y-suplementos?orde...,202102030004,1,0,google / organic,2,1,0,0,1.1.0.0
32472,Caudalie Agua Refrescante Rose de Vigne,/es/protectores-solares/4402-heliocare-360-wat...,202101311829,1,0,(direct) / (none),1,1,0,0,1.1.0.0


In [235]:
## observaciones 0.0.1.1 en las que el usuario ve el producto en su url especifica y lo añade al carrito

obs_0011 = df[df['hits_combinations']=="0.0.1.1"]
obs_0011.sample(50)

# en efecto, como en el caso de 0.0.1.0, parece que todos las observaciones tienen lugar en una ficha de producto especifica.

,ga:productName,ga:pagePath,ga:dateHourMinute,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart,hits_combinations
87756,Gema Herrerias Crema Función Barrera Diaria 50 ml,/es/productos-gema-herrerias/6751-gema-herreri...,202101292028,2,65,google / cpc,0,0,1,1,0.0.1.1
1419,▷▷ GUAM - FANGO GEL FIR ACCION 300 ml ANTICEL...,/es/anticeluliticos/4265-▷▷-guam-fango-gel-fir...,202101312249,3,61,(direct) / (none),0,0,1,1,0.0.1.1
5124,12 AZELAIC-N Gema Herrerias 30 ML,/es/productos-gema-herrerias/4477-gh-12-azelai...,202101301413,4,0,(direct) / (none),0,0,1,1,0.0.1.1
87175,Gema Herrerias AZA-GZ crema multifunción | El ...,/es/productos-gema-herrerias/7574-gh-aza-gz-cr...,202102020720,1,0,google / organic,0,0,1,1,0.0.1.1
42054,Colonia Infantil Suavinex | Olor a flores Blancas,/es/perfumes-y-fragancias/1305-suavinex-coloni...,202102031340,1,0,google / organic,0,0,4,1,0.0.1.1
46289,Dr. Arthouros Alba Contorno de Ojos ALTA POTENCIA,/es/contorno-ojos/7821-dr-arthouros-alba-conto...,202101281625,4,0,instagram.com / referral,0,0,1,1,0.0.1.1
69777,Farma Dorsch Contorno Ojos Your Eyes de Fridda...,/es/contorno-de-ojos-para-ojera-oscura/1313-fa...,202102022025,1,0,google / cpc,0,0,1,1,0.0.1.1
35226,CAUDALIE RESVERATROL LIFT CREMA CACHEMIR 50ML,/es/inicio/4149-caudalie-resveratrol-lift-crem...,202102031005,1,0,(direct) / (none),0,0,1,1,0.0.1.1
31728,"CARMEX PROTECTOR LABIAL STICK 4,25GR",/es/anticeluliticos/2834-carmex-protector-labi...,202101301408,1,0,google / organic,0,0,1,1,0.0.1.1
25664,"Biopel - Crema Hidratante , Calma, alivia y pr...",/es/piel-irritada/7717-biopel-crema-hidratante...,202101312247,1,0,google / cpc,0,0,3,1,0.0.1.1


In [238]:
# en este caso, el producto se añade al carrito pero no hay rastro de otro tipo de acciones ya sea de visualización o de click
# veamos a que se puede referir:

obs_0001 = df[df['hits_combinations']=="0.0.0.1"]
obs_0001.sample(50)

# en los datos no parece haber ninguna razón aparente que justifique la falta de visualizacion en listas o fichas de producto
# El pagepath en el que se añade el producto al carrito parece coincidir bien con la ficha de producto, bien con una lista en la que se encuentra el producto
# Tampoco hay nada particular en los canales de adquisición o en el tipo de usuario. 
# Puede ser un problema de configuracion de tags del GA de la página
# Dado que se trata del 0,26% de las observaciones (262 sobre 100000) proseguiremos con el estudio de las variables para evaluarlo más adelante.


,ga:productName,ga:pagePath,ga:dateHourMinute,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart,hits_combinations
43419,Crema Funcion Barrera EDICION LIMITADA Gema He...,/es/productos-gema-herrerias/7845-crema-funcio...,202102032040,1,0,google / cpc,0,0,0,1,0.0.0.1
96584,Heliocare 360 Pigment Solution Fluid SPF 50,/es/protectores-solares/6693-heliocare-360-pig...,202101291224,1,0,google / cpc,0,0,0,1,0.0.0.1
5856,5PUNTO5 Tónico Equilibrante | El mejor tónico...,/es/google-shopping/1635-5punto5-tonico-equili...,202101301335,1,0,google / organic,0,0,0,1,0.0.0.1
95918,Heliocare 360 Color Gel Oil Free Bronze,/es/protectores-solares/994-heliocare-360-colo...,202102032304,1,0,google / organic,0,0,0,1,0.0.0.1
16710,APIVITA MASCARILLA TONIFICANTE PARA CABELLO DE...,/es/module/iqitsearch/searchiqit?order=product...,202101281814,13,0,google / organic,0,0,0,1,0.0.0.1
90827,GH función global tratamiento intensivo para c...,/es/productos-gema-herrerias/4892-gh-funcion-g...,202101291312,2,0,google / organic,0,0,0,1,0.0.0.1
1417,▷▷ GUAM - FANGO GEL FIR ACCION 300 ml ANTICEL...,/es/anticeluliticos/4265-▷▷-guam-fango-gel-fir...,202101302229,1,0,google / organic,0,0,0,1,0.0.0.1
13915,Antiage Hair Boost Fridda Dorsch,/es/,202102030930,3,9,google / cpc,0,0,0,1,0.0.0.1
43445,Crema Funcion Barrera EDICION LIMITADA Gema He...,/es/productos-gema-herrerias/7845-crema-funcio...,202102032330,1,0,google / cpc,0,0,0,1,0.0.0.1
35255,Caudalie Resveratrol Lift Crema Cachemire mini,/es/4192-resveratrol-lift-de-caudalie.html,202101311347,1,0,google / cpc,0,0,0,1,0.0.0.1


In [239]:
# en este caso, se ha registrado un click sobre el producto en una lista pero no hay rastro de visualización.
# veamos a que se puede referir:

obs_0100 = df[df['hits_combinations']=="0.1.0.0"]
obs_0100.sample(50)

# como en el caso anterior, no hay nada aparente que pueda justificar la falta de registros en productlistviews
# dado que su representación sobre el total de representaciones tampoco es significativa (0,25%) , lo evaluaremos más tarde. 


,ga:productName,ga:pagePath,ga:dateHourMinute,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart,hits_combinations
18661,Bakuchiol Peptidos MEDIK8 Alternativa al Retin...,/es/module/iqitsearch/searchiqit,202102011844,3,0,sendinblue / email,0,1,0,0,0.1.0.0
72007,Farma Dorsch Leche corporal ultrareafirmante 4...,/es/anticeluliticos/1316-anticelulitica-mesore...,202101311723,1,0,google / cpc,0,1,0,0,0.1.0.0
96554,Heliocare 360 Pigment Solution Fluid SPF 50,/es/protectores-solares/4402-heliocare-360-wat...,202101311324,1,0,(direct) / (none),0,1,0,0,0.1.0.0
44588,CUMLAUDE LAB: GYNELAUDE HIGIENE INTIMA DIARIA GE,/es/limpiadores/7331-esthederm-osmoclean-crema...,202102041105,1,0,google / cpc,0,1,0,0,0.1.0.0
48501,Dr. Arthouros Alba Retexturizante Retinol 0.5 ...,/es/manchas/7822-dr-arthouros-alba-serum-despi...,202102010955,1,0,google / organic,0,1,0,0,0.1.0.0
17127,Aquilea Pack Articulaciones Colágeno + Magnesi...,/es/module/iqitsearch/searchiqit?page=3,202101280231,2,37,google / organic,0,1,0,0,0.1.0.0
19955,"Barra Labios ROSA ""a tope de power "" Cristina ...",/es/productos-gema-herrerias/6446-emulsion-jap...,202101290001,1,0,google / organic,0,1,0,0,0.1.0.0
39562,Champú Go Organic PELO GRASO Anticaida Farma D...,/es/contorno-de-ojos-para-ojera-oscura/1313-fa...,202101312114,3,2,google / cpc,0,1,0,0,0.1.0.0
48047,Dr. Arthouros Alba Formula Hidratación y Regen...,/es/top-ventas/6337-skinperfection-bluevert-co...,202101282210,17,0,google / organic,0,1,0,0,0.1.0.0
93427,Goah Clinic Celulit 60 CAPS,/es/vitaminas-minerales-y-suplementos/6321-eli...,202102012115,9,0,google / cpc,0,1,0,0,0.1.0.0


In [240]:
# observaciones 1.0.0.1
# el usuario ve el producto en una lista y lo añade directamente al carrito desde la misma lista:

obs_1001 = df[df['hits_combinations']=="1.0.0.1"]
obs_1001.sample(50)

# mayoritariamente en paginas de categorias de producto o resultados de busqueda
# también en la lista de otras recomendaciones de una ficha de otro producto


,ga:productName,ga:pagePath,ga:dateHourMinute,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart,hits_combinations
92168,GH LIMPIADOR DESMAQUILLANTE NATURAL 2 EN 1 (20...,/es/module/iqitsearch/searchiqit,202102022309,23,7,(direct) / (none),1,0,0,1,1.0.0.1
86950,Gema Herrerias AZA-GZ crema multifunción | El ...,/es/brand/356-gema-herrerias,202102021006,2,0,google / organic,1,0,0,1,1.0.0.1
88356,Gema Herrerias Gh 10 Antiox-VIS,/es/brand/356-gema-herrerias,202102032124,9,0,google / organic,1,0,0,1,1.0.0.1
43079,Crema Funcion Barrera EDICION LIMITADA Gema He...,/es/902-novedades,202101312235,16,3,l.instagram.com / referral,1,0,0,1,1.0.0.1
74079,Filorga hydra-filler mask.Mascarilla Superhidr...,/es/905-piel-seca?order=product.position.asc&r...,202101281818,13,0,google / organic,1,0,0,1,1.0.0.1
78062,Filorga time-filler mask.Crema Perfeccionadora...,/es/905-piel-seca?order=product.position.asc&r...,202101281818,13,0,google / organic,1,0,0,1,1.0.0.1
70079,Farma Dorsch Crema Antiarrugas Intensiva Vitam...,/es/903-top-ventas?page=2,202101300652,2,5,google / organic,1,0,0,1,1.0.0.1
39469,Champú Go Organic PELO GRASO Anticaida Farma D...,/es/brand/331-fridda-dorsch-farma-dorsch,202101310209,1,0,google / organic,1,0,0,1,1.0.0.1
11775,Agua de colonia Fridda Dorsch | Olor a fresco ...,/es/module/iqitsearch/searchiqit,202101310046,5,0,google / organic,1,0,0,1,1.0.0.1
47574,Dr. Arthouros Alba Formula Hidratación y Regen...,/es/manchas/7822-dr-arthouros-alba-serum-despi...,202102010627,4,0,google / organic,1,0,0,2,1.0.0.1


In [245]:
# observaciones 1010
# el producto se visualiza en una lista y en una ficha de producto a la vez

obs_1010 = df[df['hits_combinations']=="1.0.1.0"]
obs_1010

# Parece que este tipo de comportamiento están relacionados con determinadas referencias de producto (APIVITA limpiador, Caudalie Vinopure, Goah Clinic...)
# En todos los casos, se da en la url de la ficha de producto asi que es posible que por alguna razón, cuando el usuario está en la ficha de estos productos en la lista de recomendación se le recomienda el mismo prod
# Ya veremos más adelante si afecta a nuestro análisis 

,ga:productName,ga:pagePath,ga:dateHourMinute,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart,hits_combinations
12304,Ampollas Faciales Delapiel Renovación 15 ampollas,/es/cuidado-corporal/1265-delapiel-renovacion-...,202101280028,1,0,google / organic,1,0,1,0,1.0.1.0
12309,Ampollas Faciales Delapiel Renovación 15 ampollas,/es/cuidado-corporal/1265-delapiel-renovacion-...,202102021009,1,0,idealo.es / referral,1,0,2,0,1.0.1.0
16163,APIVITA CUERO CABELLUDO SENSIBLE con lavanda y...,/es/inicio/6156-champú-para-cuero-cabelludo-se...,202101280102,1,0,google / cpc,1,0,3,0,1.0.1.0
16356,Apivita gel limpiador pieles grasa/mixta con p...,/es/limpiadores/6133-gel-limpiador-pieles-gras...,202101282109,32,0,google / organic,1,0,1,0,1.0.1.0
16358,Apivita gel limpiador pieles grasa/mixta con p...,/es/limpiadores/6133-gel-limpiador-pieles-gras...,202101311033,20,1,IGShopping / Social,1,0,1,0,1.0.1.0
16359,Apivita gel limpiador pieles grasa/mixta con p...,/es/limpiadores/6133-gel-limpiador-pieles-gras...,202101312217,4,0,google / organic,1,0,1,0,1.0.1.0
16360,Apivita gel limpiador pieles grasa/mixta con p...,/es/limpiadores/6133-gel-limpiador-pieles-gras...,202102021825,1,0,google / organic,1,0,1,0,1.0.1.0
16362,Apivita gel limpiador pieles grasa/mixta con p...,/es/limpiadores/6133-gel-limpiador-pieles-gras...,202102031404,1,0,(direct) / (none),1,0,1,0,1.0.1.0
36866,Caudalie Vinopure Gel Limpiador 150 ml,/es/limpiadores/4935-friendly-url-autogenerati...,202101301529,9,0,google / organic,1,0,1,0,1.0.1.0
36868,Caudalie Vinopure Gel Limpiador 150 ml,/es/limpiadores/4935-friendly-url-autogenerati...,202101311721,2,0,google / organic,1,0,1,0,1.0.1.0


In [247]:
# observaciones 1.1.0.1
# en una lista, el usuario da click y añade el prod al carrito

obs_1101 = df[df['hits_combinations']=="1.1.0.1"]
obs_1101

# solo existe un caso, pero en efecto corresponde con un listado en una pagina de una categoria en la que es posible que en el transcurso del mismo minuto el usuario hiciera las 2 acciones:click y añadir al carrito


,ga:productName,ga:pagePath,ga:dateHourMinute,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart,hits_combinations
49213,Dr. Arthouros Alba Serum Despigmentante Amplio...,/es/909-piel-manchas?order=product.price.desc&...,202102031440,1,0,cantabrialabs.es / referral,2,1,0,1,1.1.0.1


In [248]:
# en este caso se visualiza el producto a la vez en ficha de producto y en una lista, pinchando ademas el usuario sobre el producto
# resulta algo inconsistente dado que ademas este evento tiene lugar en una pagina tipo lista en el que se muestran varios productos  de una misma marca
# dado que solo es un caso entre 100.000 lo omitiremos por el moment

obs_1110 = df[df['hits_combinations']=="1.1.1.0"]
obs_1110

,ga:productName,ga:pagePath,ga:dateHourMinute,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart,hits_combinations
1929,▷▷Emulsión limpiadora JAPONESA Kire Gema Herre...,/es/brand/356-gema-herrerias,202101311750,2,0,google / organic,1,1,1,0,1.1.1.0


In [249]:
# no se visualiza el producto en una lista, aun tratandose de una url tipo lista,  pero se hace click igualmente como si lo estuviera
# no parece tener sentido, pero como en el caso anterior lo omitiremos de momento por tratarse de un caso aislado.

obs_0110 = df[df['hits_combinations']=="0.1.1.0"]
obs_0110

,ga:productName,ga:pagePath,ga:dateHourMinute,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart,hits_combinations
43179,Crema Funcion Barrera EDICION LIMITADA Gema He...,/es/brand/356-gema-herrerias,202101311750,2,0,google / organic,0,2,1,0,0.1.1.0


En cualquier caso, podemos concluir, que la consulta actual captura correctamente todos los posibles comportamientos del usuario con respecto al producto y la conversión. 

### Sampling data

Uno de los principales inconveniente de Google Analytics es que para algunos informes, especialmente los personalizados como es nuestro caso, devuelve datos 'sampleados' con lo que perdemos predicción en nuestro análisis. Es frecuente que al pedir un mismo informe varias veces obtengamos resultados diferentes. 

Generalmente, se produce cuando hacemos cualquier tipo de consulta específica que trabaje con más de 500.000 sesiones (por web property) o cuando solicitamos un informe que contiene más de 1.000.000 de valores de dimensión (resultados). Por ejemplo, si solicitamos un informe de contenido que nos debiera de devolver 1.000.000 URLs únicas, Analytics divide este dato entre el rango de días aplicado. Si es de un mes entonces tendríamos unas 30.000 URLs únicas (1.000.000/30).

Para descartar que esto no esté sucediendo en la consulta, vamos a pedir que devuelva en el informe el número de muestras leídas y presentes.

> reportData.samplesReadCounts: If the results are sampled, this returns the total number of samples read, one entry per date range. If the results are not sampled this field will not be defined.

> reportData.samplingSpaceSizes: If the results are sampled, this returns the total number of samples present, one entry per date range. If the results are not sampled this field will not be defined.

Ver "muestras de datos" en:https://developers.google.com/analytics/devguides/reporting/core/v4/migration?hl=es

In [87]:
#REPORT REQUEST:
response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'7daysAgo','endDate':'today'}],
        'metrics':
        [
            {'expression':'ga:productListViews'},
            {'expression':'ga:productListClicks'},
            {'expression':'ga:productDetailViews'},
            {'expression':'ga:productAddsToCart'},
        ],
        'dimensions':
        [
            {'name':'ga:productName'},
            {'name':'ga:pagePath'},
            {'name':'ga:dateHourMinute'},
            {'name':'ga:sessionCount'},
            {'name':'ga:daysSinceLastSession'},
            {'name':'ga:sourceMedium'},
            
        ],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        #'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        'pageSize':100000
    }]
}).execute()


#PARSING THE REPORT DATA:
list_ = []

for report in response.get('reports',[]):
    
    columnHeader = report.get('columnHeader',{})
    dimensionHeaders = columnHeader.get('dimensions',[])
    metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])
    
    rows = report.get('data',{}).get('rows',[])
    for row in rows:
        dict_={}
        dimensions = row.get('dimensions',[])
        dateRangeValues=row.get('metrics',[])
        
        for header, dimension in zip(dimensionHeaders,dimensions):
            dict_[header]=dimension
        
        for i,values in enumerate (dateRangeValues):
            for metricHeader,value in zip(metricHeaders,values.get('values')):
                if ',' in value or '.' in value:
                  dict_[metricHeader.get('name')] = float(value)
                else:
                  dict_[metricHeader.get('name')] = int(value)
                
        list_.append(dict_)
    
    # Sampled Data? : 
    sampling = False
    sample_sizes = report.get('data',{}).get('samplesReadCounts',[])
    sample_spaces = report.get('data',{}).get('samplingSpaceSizes',[])
    if sample_sizes and sample_spaces:
        sampling = True
        
#DATAFRAME WITH DATA:
df = pd.DataFrame(list_)
print(df.shape,sample_sizes,sample_spaces,sampling)
df.head()

(100000, 10) [] [] False


,ga:productName,ga:pagePath,ga:dateHourMinute,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart
0,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1041-neostrata-basis-redox-crema-antiarrug...,202101310813,3,0,google / organic,1,0,0,0
1,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1041-neostrata-basis-redox-crema-antiarrug...,202101311714,2,0,(direct) / (none),1,0,0,0
2,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1041-neostrata-basis-redox-crema-antiarrug...,202101312017,5,0,google / organic,1,0,0,0
3,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1041-neostrata-basis-redox-crema-antiarrug...,202101312034,5,0,google / organic,1,0,0,0
4,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1231-mia-triple-5-11ml-8436558880467.html,202102021836,1,0,google / cpc,1,0,0,0


Los campos 'samplesReadCounts' y 'samplingSpaceSizes' no se han definido en el informe de respuesta asi que nuestra consulta no devuelve datos sampleados.

###  Max 100.000 rows per report request

Otras de las limitationes con las que nos encontramos al trabajar con la API de Google Analytics es que únicamente podemos extraer 100.000 filas por informe.

En este sentido si el resultado de nuestra consulta es superior a 100.000 filas, es necesario realizar varias calls, cada una de ellas para una de las páginas del informe. 

Empezaremos por extraer de nuestro informe, 'nextPageToken' que nos indica el indice (número de fila) a partir del cual debe extraer los datos en la siguiente consulta. Deberemos incorporar este valor en el parametro pagetoken de la siguiente call.

In [133]:
#REPORT REQUEST:

response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'2021-01-01','endDate':'2021-01-07'}],
        'metrics':
        [
            {'expression':'ga:productListViews'},
            {'expression':'ga:productListClicks'},
            {'expression':'ga:productDetailViews'},
            {'expression':'ga:productAddsToCart'},
        ],
        'dimensions':
        [
            {'name':'ga:productName'},
            {'name':'ga:pagePath'},
            {'name':'ga:dateHourMinute'},
            {'name':'ga:sessionCount'},
            {'name':'ga:daysSinceLastSession'},
            {'name':'ga:sourceMedium'},   
            
        ],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        #'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        'pageSize':100000 #if we do not specify we will get 1000 rows
    }]
}).execute()

#PARSING THE REPORT DATA:
list_ = []

for report in response.get('reports',[]):
    
    columnHeader = report.get('columnHeader',{})
    dimensionHeaders = columnHeader.get('dimensions',[])
    metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])

    rows = report.get('data',{}).get('rows',[])
    for row in rows:
        dict_={}
        dimensions = row.get('dimensions',[])
        dateRangeValues=row.get('metrics',[])
        
        for header, dimension in zip(dimensionHeaders,dimensions):
            dict_[header]=dimension
        
        for i,values in enumerate (dateRangeValues):
            for metricHeader,value in zip(metricHeaders,values.get('values')):
                if ',' in value or '.' in value:
                  dict_[metricHeader.get('name')] = float(value)
                else:
                  dict_[metricHeader.get('name')] = int(value)
                
        list_.append(dict_)
        
    # Sampled Data? : 
    sampling = "No sampled data"
    sample_sizes = report.get('data',{}).get('samplesReadCounts',[])
    sample_spaces = report.get('data',{}).get('samplingSpaceSizes',[])
    if sample_sizes and sample_spaces:
        sampling = "Sampled data"
        
    # NextPageToken for following queries:
    next_page_token = report.get('nextPageToken',None)

#DATAFRAME WITH DATA:
import pandas as pd
df_p1= pd.DataFrame(list_)
print(df_p1.shape, sampling, next_page_token)
df_p1.head()

(100000, 10) No sampled data 100000


,ga:productName,ga:pagePath,ga:dateHourMinute,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart
0,(not set),/es/2-inicio,202101072258,2,13,youtube.com / referral,1,0,0,0
1,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1041-neostrata-basis-redox-crema-antiarrug...,202101031955,4,1,google / cpc,1,0,0,0
2,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1086-neostrata-skin-active-cellular-restor...,202101050039,3,0,cantabrialabs.es / referral,1,0,0,0
3,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1086-neostrata-skin-active-cellular-restor...,202101050059,3,0,cantabrialabs.es / referral,1,0,0,0
4,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/4567-medik8-super-c30-intense-30-ml-818625...,202101062119,1,0,google / cpc,1,0,0,0


Tal y como he indicado anteriormente, para la segunda consulta, indicamos el parametro 'pageToken' con el valor que nos ha devuelto el informe anterior para nextPageToken. En definitiva, especificamos a partir de que fila tiene que extraer los datos en esta ocasión:

In [138]:
#REPORT REQUEST:

response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'2021-01-01','endDate':'2021-01-07'}],
        'metrics':
        [
            {'expression':'ga:productListViews'},
            {'expression':'ga:productListClicks'},
            {'expression':'ga:productDetailViews'},
            {'expression':'ga:productAddsToCart'},
        ],
        'dimensions':
        [
            {'name':'ga:productName'},
            {'name':'ga:pagePath'},
            {'name':'ga:dateHourMinute'},
            {'name':'ga:sessionCount'},
            {'name':'ga:daysSinceLastSession'},
            {'name':'ga:sourceMedium'},   
            
        ],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        #'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        "pageToken": "100000", #extract data from 100.000 index 
        'pageSize':100000 #if we do not specify we will get 1000 rows
    }]
}).execute()

#PARSING THE REPORT DATA:
list_ = []

for report in response.get('reports',[]):
    
    columnHeader = report.get('columnHeader',{})
    dimensionHeaders = columnHeader.get('dimensions',[])
    metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])

    rows = report.get('data',{}).get('rows',[])
    for row in rows:
        dict_={}
        dimensions = row.get('dimensions',[])
        dateRangeValues=row.get('metrics',[])
        
        for header, dimension in zip(dimensionHeaders,dimensions):
            dict_[header]=dimension
        
        for i,values in enumerate (dateRangeValues):
            for metricHeader,value in zip(metricHeaders,values.get('values')):
                if ',' in value or '.' in value:
                  dict_[metricHeader.get('name')] = float(value)
                else:
                  dict_[metricHeader.get('name')] = int(value)
                
        list_.append(dict_)
        
    # Sampled Data? : 
    sampling = "No sampled data"
    sample_sizes = report.get('data',{}).get('samplesReadCounts',[])
    sample_spaces = report.get('data',{}).get('samplingSpaceSizes',[])
    if sample_sizes and sample_spaces:
        sampling = "Sampled data"
        
    # NextPageToken for following queries:
    next_page_token = report.get('nextPageToken',None)

#DATAFRAME WITH DATA:
import pandas as pd
df_p2= pd.DataFrame(list_)
print(df_p2.shape, sampling, next_page_token)
df_p2.head()

(100000, 10) No sampled data 200000


,ga:productName,ga:pagePath,ga:dateHourMinute,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart
0,La Roche Posay Desodarante en Stick Sin aluminio,/es/module/iqitsearch/searchiqit,202101032049,1,0,google / cpc,1,0,0,0
1,La Roche Posay Desodarante en Stick Sin aluminio,/es/module/iqitsearch/searchiqit,202101032329,3,0,google / organic,1,0,0,0
2,La Roche Posay Desodarante en Stick Sin aluminio,/es/module/iqitsearch/searchiqit,202101042306,9,0,google / organic,1,0,0,0
3,La Roche Posay Desodarante en Stick Sin aluminio,/es/module/iqitsearch/searchiqit,202101061027,9,35,sendinblue / email,1,0,0,0
4,La Roche Posay Desodarante en Stick Sin aluminio,/es/module/iqitsearch/searchiqit,202101070828,5,0,google / cpc,1,0,0,0


En efecto, la primera y la segunda consulta devuelven dataframes diferentes. Parece que la primera y la segunda página del informe se han descargado con éxito.

Siguiendo la misma lógica anterior, relanzamos una tercera consulta, indicando esta vez como pagetoken 200000, que es el valor que nos ha devuelto nextPageToken en la consulta anterior:

In [139]:
response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'2021-01-01','endDate':'2021-01-07'}],
        'metrics':
        [
            {'expression':'ga:productListViews'},
            {'expression':'ga:productListClicks'},
            {'expression':'ga:productDetailViews'},
            {'expression':'ga:productAddsToCart'},
        ],
        'dimensions':
        [
            {'name':'ga:productName'},
            {'name':'ga:pagePath'},
            {'name':'ga:dateHourMinute'},
            {'name':'ga:sessionCount'},
            {'name':'ga:daysSinceLastSession'},
            {'name':'ga:sourceMedium'},   
            
        ],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        #'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        "pageToken": "200000", #extract data from 100.000 index 
        'pageSize':100000 #if we do not specify we will get 1000 rows
    }]
}).execute()

#PARSING THE REPORT DATA:
list_ = []

for report in response.get('reports',[]):
    
    columnHeader = report.get('columnHeader',{})
    dimensionHeaders = columnHeader.get('dimensions',[])
    metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])

    rows = report.get('data',{}).get('rows',[])
    for row in rows:
        dict_={}
        dimensions = row.get('dimensions',[])
        dateRangeValues=row.get('metrics',[])
        
        for header, dimension in zip(dimensionHeaders,dimensions):
            dict_[header]=dimension
        
        for i,values in enumerate (dateRangeValues):
            for metricHeader,value in zip(metricHeaders,values.get('values')):
                if ',' in value or '.' in value:
                  dict_[metricHeader.get('name')] = float(value)
                else:
                  dict_[metricHeader.get('name')] = int(value)
                
        list_.append(dict_)
        
    # Sampled Data? : 
    sampling = "No sampled data"
    sample_sizes = report.get('data',{}).get('samplesReadCounts',[])
    sample_spaces = report.get('data',{}).get('samplingSpaceSizes',[])
    if sample_sizes and sample_spaces:
        sampling = "Sampled data"
        
    # NextPageToken for following queries:
    next_page_token = report.get('nextPageToken',None)

#DATAFRAME WITH DATA:
import pandas as pd
df_p3= pd.DataFrame(list_)
print(df_p3.shape, sampling, next_page_token)
df_p3.head()

(14562, 10) No sampled data None


,ga:productName,ga:pagePath,ga:dateHourMinute,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart
0,SVR SENSIFINE Bálsamo Desmaquillante 100ml,/es/emulsiones-y-desmaquillantes-/6981-svr-sen...,202101072103,1,0,google / organic,0,0,3,0
1,SVR SENSIFINE Bálsamo Desmaquillante 100ml,/es/emulsiones-y-desmaquillantes-/6981-svr-sen...,202101072106,1,0,google / organic,0,0,1,0
2,SVR SENSIFINE Bálsamo Desmaquillante 100ml,/es/emulsiones-y-desmaquillantes-/6981-svr-sen...,202101072217,1,0,google / organic,0,0,3,0
3,SVR SENSIFINE Bálsamo Desmaquillante 100ml,/es/emulsiones-y-desmaquillantes-/6981-svr-sen...,202101072311,7,0,sendinblue / email,0,0,1,0
4,SVR SENSIFINE Bálsamo Desmaquillante 100ml,/es/emulsiones-y-desmaquillantes-/7420-skinceu...,202101021220,9,3,m.facebook.com / referral,1,0,0,0


Esta tercera consulta, devuelve 14.562 filas y para el campo "next_page_token" devuelve None, por lo que con esta y las 2 consultas anteriores ya habríamos extraido la totalidad de las filas.  

Dado que se trata de un proceso iterativo, incluiremos un bucle "for" a nuestro request que nos permita extraer la totalidad de los datos ejecutando unicamente una vez la celda:

In [142]:
pagetokens = [None] + [str(x) for x in range(0,1100000,100000)][1:]
list_ = [] # empty list for row data
sampled_data = [] #empty list for sampling results
next_page_token_list = [] #empty list for next_page_tokens in each iteration
full_data_loaded = "All the data were loaded for this period and request" #Control message

for pagetoken in pagetokens:

    #REPORT REQUEST:
    response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'2021-01-01','endDate':'2021-01-07'}],
        'metrics':
        [
            {'expression':'ga:productListViews'},
            {'expression':'ga:productListClicks'},
            {'expression':'ga:productDetailViews'},
            {'expression':'ga:productAddsToCart'},
        ],
        'dimensions':
        [
            {'name':'ga:productName'},
            {'name':'ga:pagePath'},
            {'name':'ga:dateHourMinute'},
            {'name':'ga:sessionCount'},
            {'name':'ga:daysSinceLastSession'},
            {'name':'ga:sourceMedium'},     
        ],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        #'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        "pageToken": pagetoken, #extract data from pagetoken index
        'pageSize':100000 #if we do not specify we will get 1000 rows
    }]
}).execute()

    #PARSING THE REPORT DATA:
    
    for report in response.get('reports',[]):
        
        columnHeader = report.get('columnHeader',{})
        dimensionHeaders = columnHeader.get('dimensions',[])
        metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])
        
        rows = report.get('data',{}).get('rows',[])
        for row in rows:
            dict_={}
            dimensions = row.get('dimensions',[])
            dateRangeValues=row.get('metrics',[])
            
            for header, dimension in zip(dimensionHeaders,dimensions):
                dict_[header]=dimension
            
            for i,values in enumerate (dateRangeValues):
                for metricHeader,value in zip(metricHeaders,values.get('values')):
                    if ',' in value or '.' in value:
                      dict_[metricHeader.get('name')] = float(value)
                    else:
                      dict_[metricHeader.get('name')] = int(value)
                    
            list_.append(dict_)

        # Sampled Data? : 
        sampling = "No sampled data"
        sample_sizes = report.get('data',{}).get('samplesReadCounts',[])
        sample_spaces = report.get('data',{}).get('samplingSpaceSizes',[])
        if sample_sizes and sample_spaces:
            sampling = "Sampled data"
        sampled_data.append(sampling)
        
        # NextPageToken for following queries:    
        next_page_token = report.get('nextPageToken',None)
        next_page_token_list.append(str(next_page_token))
        
    if next_page_token == None: #to stop iterations once all the data are loaded
        break
    if int(next_page_token) > 1000000: #to warn about remaining pages
        full_data_loaded = "Remaining data to request. Please update pagetokens variable."

#DATAFRAME WITH DATA:
import pandas as pd
df= pd.DataFrame(list_)
print((6*"*")+"Results of our request"+(6*"*"))
print("Shape of the final dataframe: " + str(df.shape))
print("Number of requests or iterations: " + str(len(next_page_token_list)))
print("Sampling for each iteration: " + str(sampled_data))
print("Next page token for each iteration: " + str(next_page_token_list))
print("Were all data exported? " + full_data_loaded)
print("Five first rows of our dataframe:")
df.head()

******Results of our request******
Shape of the final dataframe: (214562, 10)
Number of requests or iterations: 3
Sampling for each iteration: ['No sampled data', 'No sampled data', 'No sampled data']
Next page token for each iteration: ['100000', '200000', 'None']
Were all data exported?All the data were loaded for this period and request
Five first rows of our dataframe:


,ga:productName,ga:pagePath,ga:dateHourMinute,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart
0,(not set),/es/2-inicio,202101072258,2,13,youtube.com / referral,1,0,0,0
1,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1041-neostrata-basis-redox-crema-antiarrug...,202101031955,4,1,google / cpc,1,0,0,0
2,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1086-neostrata-skin-active-cellular-restor...,202101050039,3,0,cantabrialabs.es / referral,1,0,0,0
3,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1086-neostrata-skin-active-cellular-restor...,202101050059,3,0,cantabrialabs.es / referral,1,0,0,0
4,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/4567-medik8-super-c30-intense-30-ml-818625...,202101062119,1,0,google / cpc,1,0,0,0


### Max 7 dimensions and 10 metrics per report request

Centremonos ahora en otros features y dimensiones que puedan explicar nuestro target.

De cara a nuestro modelo, deberemos considerar:
- Features de producto para cada observación (i.e.: marca, categoria, precio, descuentos...)
- Features temporales para cada observación (i.e.: día de la semana, mes, horas...)
- Features de usuario para cada observación (i.e.: ciudad, franja de edad, tipo de dispositivo, sistema operativo...)
- Features de URL para cada observación (i.e.: tipo de página, page depth, tiempo de carga de la página, posición del producto en la lista...)
- Features de sesión para cada observación (i.e.:landing page, previous page, session duration...)
- Features de fuentes de adquisión para cada observación (i.e.: source, medium, campaign...)

Ahora bien, otra de las restricciones que impone Google Analytics es que no se pueden exportar más de 7 dimensiones (equivalentes a variables descriptivas) y 10 métricas (variables cuantitativas) en una misma request. Asimismo, algunas de las metrícas y dimensiones son incompatibles entre sí. 

Por esta razón, habrá que exportar varios informes que luego uniremos con un join. 

Para poder hacer el join, tendremos que utilizar campos comunes que nos permitan tambien identificar cada observacion como un usuario. Intentaremos utilizar el mínimo de dimensiones y métricas que garanticen su identificacion para utilizar el resto de dimensiones y métricas disponibles para exportar nuevos features para nuestro modelo. 

Para la primera prueba tomaremos como dimensiones y métricas comunes:
- productName (dimension)
- dateHourMinute (dimension)
- pagePath (dimension)
- productListViews (metric)
- productListClicks (metric)
- productDetailViews (metric)
- productAddsToCart (metric)

En una primera consulta, solicitaremos a mayores las siguientes dimensiones:
- sessionCount
- daysSinceLastSession
- sourceMedium
- pageDepth

Y en una segunda consulta, añadiremos 4 dimensiones diferentes:
- city
- deviceCategory
- operatingSystem
- campaign

Una vez tengamos ambos informes para el mismo periodo, veremos si al fusionar unos con otros con la función merge(), todas las observaciones quedan emparejadas y nuestros campos comunes son suficientes.


In [81]:
# FIRST REQUEST:

pagetokens = [None] + [str(x) for x in range(0,1100000,100000)][1:]
list_ = [] # empty list for row data
sampled_data = [] #empty list for sampling results
next_page_token_list = [] #empty list for next_page_tokens in each iteration
full_data_loaded = "All the data were loaded for this period and request" #Control message

for pagetoken in pagetokens:

    #REPORT REQUEST:
    response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'2021-01-01','endDate':'2021-01-07'}],
        'metrics':
        [
            {'expression':'ga:productListViews'},
            {'expression':'ga:productListClicks'},
            {'expression':'ga:productDetailViews'},
            {'expression':'ga:productAddsToCart'},
        ],
        'dimensions':
        [
            {'name':'ga:productName'},
            {'name':'ga:pagePath'},
            {'name':'ga:dateHourMinute'},
            {'name':'ga:sessionCount'},
            {'name':'ga:daysSinceLastSession'},
            {'name':'ga:sourceMedium'},
            {'name':'ga:pageDepth'},
        ],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        #'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        "pageToken": pagetoken, #extract data from pagetoken index
        'pageSize':100000 #if we do not specify we will get 1000 rows
    }]
}).execute()

    #PARSING THE REPORT DATA:
    
    for report in response.get('reports',[]):
        
        columnHeader = report.get('columnHeader',{})
        dimensionHeaders = columnHeader.get('dimensions',[])
        metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])
        
        rows = report.get('data',{}).get('rows',[])
        for row in rows:
            dict_={}
            dimensions = row.get('dimensions',[])
            dateRangeValues=row.get('metrics',[])
            
            for header, dimension in zip(dimensionHeaders,dimensions):
                dict_[header]=dimension
            
            for i,values in enumerate (dateRangeValues):
                for metricHeader,value in zip(metricHeaders,values.get('values')):
                    if ',' in value or '.' in value:
                      dict_[metricHeader.get('name')] = float(value)
                    else:
                      dict_[metricHeader.get('name')] = int(value)
                    
            list_.append(dict_)

        # Sampled Data? : 
        sampling = "No sampled data"
        sample_sizes = report.get('data',{}).get('samplesReadCounts',[])
        sample_spaces = report.get('data',{}).get('samplingSpaceSizes',[])
        if sample_sizes and sample_spaces:
            sampling = "Sampled data"
        sampled_data.append(sampling)
        
        # NextPageToken for following queries:    
        next_page_token = report.get('nextPageToken',None)
        next_page_token_list.append(str(next_page_token))
        
    if next_page_token == None: #to stop iterations once all the data are loaded
        break
    if int(next_page_token) > 1000000: #to warn about remaining pages
        full_data_loaded = "Remaining data to request. Please update pagetokens variable."

#DATAFRAME WITH DATA:
import pandas as pd
first_request= pd.DataFrame(list_)
print((6*"*")+"Results of our request"+(6*"*"))
print("Shape of the final dataframe: " + str(first_request.shape))
print("Number of requests or iterations: " + str(len(next_page_token_list)))
print("Sampling for each iteration: " + str(sampled_data))
print("Next page token for each iteration: " + str(next_page_token_list))
print("Were all data exported? " + full_data_loaded)
print("Five first rows of our dataframe:")
first_request.head()

******Results of our request******
Shape of the final dataframe: (215561, 11)
Number of requests or iterations: 3
Sampling for each iteration: ['No sampled data', 'No sampled data', 'No sampled data']
Next page token for each iteration: ['100000', '200000', 'None']
Were all data exported? All the data were loaded for this period and request
Five first rows of our dataframe:


,ga:productName,ga:pagePath,ga:dateHourMinute,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:pageDepth,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart
0,(not set),/es/2-inicio,202101072258,2,13,youtube.com / referral,4,1,0,0,0
1,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1041-neostrata-basis-redox-crema-antiarrug...,202101031955,4,1,google / cpc,10,1,0,0,0
2,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1086-neostrata-skin-active-cellular-restor...,202101050039,3,0,cantabrialabs.es / referral,49,1,0,0,0
3,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1086-neostrata-skin-active-cellular-restor...,202101050059,3,0,cantabrialabs.es / referral,49,1,0,0,0
4,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/4567-medik8-super-c30-intense-30-ml-818625...,202101062119,1,0,google / cpc,3,1,0,0,0


In [13]:
#SECOND REQUEST:

pagetokens = [None] + [str(x) for x in range(0,1100000,100000)][1:]
list_ = [] # empty list for row data
sampled_data = [] #empty list for sampling results
next_page_token_list = [] #empty list for next_page_tokens in each iteration
full_data_loaded = "All the data were loaded for this period and request" #Control message

for pagetoken in pagetokens:

    #REPORT REQUEST:
    response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'2021-01-01','endDate':'2021-01-07'}],
        'metrics':
        [
            {'expression':'ga:productListViews'},
            {'expression':'ga:productListClicks'},
            {'expression':'ga:productDetailViews'},
            {'expression':'ga:productAddsToCart'},
        ],
        'dimensions':
        [
            {'name':'ga:productName'},
            {'name':'ga:pagePath'},
            {'name':'ga:dateHourMinute'},
            {'name':'ga:city'},
            {'name':'ga:deviceCategory'},
            {'name':'ga:operatingSystem'},
            {'name':'ga:campaign'}
        ],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        #'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        "pageToken": pagetoken, #extract data from pagetoken index
        'pageSize':100000 #if we do not specify we will get 1000 rows
    }]
}).execute()

    #PARSING THE REPORT DATA:
    
    for report in response.get('reports',[]):
        
        columnHeader = report.get('columnHeader',{})
        dimensionHeaders = columnHeader.get('dimensions',[])
        metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])
        
        rows = report.get('data',{}).get('rows',[])
        for row in rows:
            dict_={}
            dimensions = row.get('dimensions',[])
            dateRangeValues=row.get('metrics',[])
            
            for header, dimension in zip(dimensionHeaders,dimensions):
                dict_[header]=dimension
            
            for i,values in enumerate (dateRangeValues):
                for metricHeader,value in zip(metricHeaders,values.get('values')):
                    if ',' in value or '.' in value:
                      dict_[metricHeader.get('name')] = float(value)
                    else:
                      dict_[metricHeader.get('name')] = int(value)
                    
            list_.append(dict_)

        # Sampled Data? : 
        sampling = "No sampled data"
        sample_sizes = report.get('data',{}).get('samplesReadCounts',[])
        sample_spaces = report.get('data',{}).get('samplingSpaceSizes',[])
        if sample_sizes and sample_spaces:
            sampling = "Sampled data"
        sampled_data.append(sampling)
        
        # NextPageToken for following queries:    
        next_page_token = report.get('nextPageToken',None)
        next_page_token_list.append(str(next_page_token))
        
    if next_page_token == None: #to stop iterations once all the data are loaded
        break
    if int(next_page_token) > 1000000: #to warn about remaining pages
        full_data_loaded = "Remaining data to request. Please update pagetokens variable."

#DATAFRAME WITH DATA:
import pandas as pd
second_request= pd.DataFrame(list_)
print((6*"*")+"Results of our request"+(6*"*"))
print("Shape of the final dataframe: " + str(second_request.shape))
print("Number of requests or iterations: " + str(len(next_page_token_list)))
print("Sampling for each iteration: " + str(sampled_data))
print("Next page token for each iteration: " + str(next_page_token_list))
print("Were all data exported? " + full_data_loaded)
print("Five first rows of our dataframe:")
second_request.head()

******Results of our request******
Shape of the final dataframe: (215589, 11)
Number of requests or iterations: 3
Sampling for each iteration: ['No sampled data', 'No sampled data', 'No sampled data']
Next page token for each iteration: ['100000', '200000', 'None']
Were all data exported? All the data were loaded for this period and request
Five first rows of our dataframe:


,ga:productName,ga:pagePath,ga:dateHourMinute,ga:city,ga:deviceCategory,ga:operatingSystem,ga:campaign,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart
0,(not set),/es/2-inicio,202101072258,Madrid,mobile,Android,(not set),1,0,0,0
1,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1041-neostrata-basis-redox-crema-antiarrug...,202101031955,(not set),mobile,Android,Resto Oportunidades,1,0,0,0
2,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1086-neostrata-skin-active-cellular-restor...,202101050039,Valladolid,tablet,iOS,(not set),1,0,0,0
3,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1086-neostrata-skin-active-cellular-restor...,202101050059,Valladolid,tablet,iOS,(not set),1,0,0,0
4,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/4567-medik8-super-c30-intense-30-ml-818625...,202101062119,Alcorcon,mobile,Android,Resto Oportunidades,1,0,0,0


En un primer vistazo, observamos que el tamaño de los informes no coincide. first_request tiene 28 filas menos que second_request.

In [14]:
first_request.shape[0]-second_request.shape[0]

-28

Hagamos el merge para ver qué observaciones difieren:

In [15]:
join_df = first_request.merge(second_request,how="outer")
join_df.shape

(226195, 15)

In [16]:
join_df.dropna().shape

(225085, 15)

El total de filas con NAN y por lo tanto que no coinciden en ambos dataframes son:

In [18]:
join_df.shape[0] - join_df.dropna().shape[0]

1110

In [19]:
# Filas con NaN:

is_NaN = join_df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = join_df[row_has_NaN]
rows_with_NaN.sample(50)

,ga:productName,ga:pagePath,ga:dateHourMinute,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:pageDepth,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart,ga:city,ga:deviceCategory,ga:operatingSystem,ga:campaign
226022,Neceser Cosmetica y Maquillaje CEKABAN NEGRO II,/es/exfoliantes/4393-cristina-mitre-gh5azelaic...,202101031713,NaN,NaN,NaN,NaN,1,0,0,0,Pamplona,mobile,Android,(not set)
226066,Pure HA Croma Mascarilla Facial con Acido Hial...,/es/903-top-ventas?page=2,202101071747,NaN,NaN,NaN,NaN,2,0,0,0,Madrid,mobile,iOS,(not set)
73474,FREZYDERM Crema Solar CON color 50- Frezyderm,/es/,202101031625,1,0,google / organic,2,1,0,0,0,NaN,NaN,NaN,NaN
225921,Loción facial exfoliante 5 AZELAIC-S Gema Herr...,/es/exfoliantes/4393-cristina-mitre-gh5azelaic...,202101071139,NaN,NaN,NaN,NaN,0,0,1,1,(not set),tablet,iOS,(not set)
222037,Unique Pink Collagen,/es/vitaminas-minerales-y-suplementos/1391-uni...,202101071109,1,0,google / cpc,2,0,0,2,0,NaN,NaN,NaN,NaN
61689,Esthederm Intensivo Retinol Sérum 30ml,/es/,202101022257,8,0,google / organic,2,1,0,0,0,NaN,NaN,NaN,NaN
226061,Pack Unique Pink Collagen y Green Collagen,/es/packs-premium/7217-pack-unique-pink-collag...,202101031802,NaN,NaN,NaN,NaN,0,0,2,0,Barcelona,mobile,iOS,(not set)
175827,RETINOL 0.3 NP Gema Herrerias Serum 0.3 Retino...,/es/top-ventas/6285-retinol-03-gema-herrerias-...,202101071451,1,0,google / cpc,1,0,0,1,0,NaN,NaN,NaN,NaN
139364,My Organics Goji Angel Potion 100 ml,/es/,202101061054,1,0,sendinblue / email,2,2,0,0,0,NaN,NaN,NaN,NaN
107743,Loción facial exfoliante 5 AZELAIC-S Gema Herr...,/es/exfoliantes/4393-cristina-mitre-gh5azelaic...,202101031846,1,0,youtube.com / referral,3,0,0,5,0,NaN,NaN,NaN,NaN


In [20]:
# Columnas con NaN:

join_df [ join_df.columns[join_df.isna().any()]]

,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:pageDepth,ga:city,ga:deviceCategory,ga:operatingSystem,ga:campaign
0,2,13,youtube.com / referral,4,Madrid,mobile,Android,(not set)
1,4,1,google / cpc,10,(not set),mobile,Android,Resto Oportunidades
2,3,0,cantabrialabs.es / referral,49,Valladolid,tablet,iOS,(not set)
3,3,0,cantabrialabs.es / referral,49,Valladolid,tablet,iOS,(not set)
4,1,0,google / cpc,3,Alcorcon,mobile,Android,Resto Oportunidades
...,...,...,...,...,...,...,...,...
226190,NaN,NaN,NaN,NaN,(not set),mobile,Android,Champions shopping
226191,NaN,NaN,NaN,NaN,(not set),mobile,iOS,(not set)
226192,NaN,NaN,NaN,NaN,(not set),mobile,iOS,Champions shopping
226193,NaN,NaN,NaN,NaN,Madrid,desktop,Windows,Champions shopping


Como es lógico, sólo tenemos NaNs en las dimensiones no coincidentes en ambos dataframes. No obstante, dado que second_request tenía más observaciones que first_request hubiera tenido sentido que solo lo tuvieran las columnas especificas de first_request. Por desgracia, no es el caso.

Veamos igualmente, cuantas filas con NaNs coinciden con nuestra primera consulta y cuantas con la segunda:

In [21]:
#number of rows with NaN in column "sessioncount" specific from first request

1110 - rows_with_NaN['ga:sessionCount'].dropna().shape[0]

524

In [22]:
#number of rows with NaN in column "city" specific from second request

1110 - rows_with_NaN['ga:city'].dropna().shape[0]

586

In [23]:
586-524

62

Vemos que 586 observaciones de second_request no han encontrado su correspondencia en first_request y 524 de first_request no lo han hecho en second_request. 

Al observar un poco más en profundidad en las filas con NaNs, se observan, en bastantes casos, valores superiores a 1 para las columnas correspondientes a hits/eventos (ListViews, List Clicks, Detail Views, Add_to_Cart)

Esto puede indicar que los datos no están suficientemente desagregados a nivel de usuario y algunas observaciones recogen información para multiples usuarios con las mismas características o valores en las dimensiones. 

Las columnas más conflictivas en este sentido son ga:productListViews y ga:productDetailViews:

In [24]:
listviews_per_obs_nan = rows_with_NaN['ga:productListViews'].value_counts()
listclicks_per_obs_nan = rows_with_NaN['ga:productListClicks'].value_counts()
detailviews_per_obs_nan = rows_with_NaN['ga:productDetailViews'].value_counts()
addscart_per_obs_nan = rows_with_NaN['ga:productAddsToCart'].value_counts()

pd.DataFrame([listviews_per_obs_nan,listclicks_per_obs_nan,detailviews_per_obs_nan,addscart_per_obs_nan]).T

,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart
0,329.0,1100.0,785.0,1086.0
1,462.0,10.0,123.0,24.0
2,276.0,NaN,117.0,NaN
3,35.0,NaN,49.0,NaN
4,6.0,NaN,23.0,NaN
5,2.0,NaN,8.0,NaN
6,NaN,NaN,1.0,NaN
7,NaN,NaN,1.0,NaN
8,NaN,NaN,1.0,NaN
9,NaN,NaN,1.0,NaN


In [25]:
listviews_per_obs = join_df['ga:productListViews'].value_counts()
listclicks_per_obs = join_df['ga:productListClicks'].value_counts()
detailviews_per_obs = join_df['ga:productDetailViews'].value_counts()
addscart_per_obs = join_df['ga:productAddsToCart'].value_counts()

pd.DataFrame([listviews_per_obs,listclicks_per_obs,detailviews_per_obs,addscart_per_obs]).T

,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart
0,17127.0,224242.0,210232.0,223837.0
1,203215.0,1926.0,11785.0,2309.0
2,5476.0,23.0,3297.0,30.0
3,322.0,1.0,806.0,10.0
4,43.0,2.0,53.0,6.0
5,11.0,NaN,16.0,1.0
6,NaN,1.0,1.0,2.0
7,NaN,NaN,1.0,NaN
8,NaN,NaN,1.0,NaN
9,1.0,NaN,2.0,NaN


In [26]:
rows_with_NaN_agg_obs = rows_with_NaN[rows_with_NaN['ga:productListViews']>=2].shape[0] + rows_with_NaN[rows_with_NaN['ga:productListClicks']>=2].shape[0]+rows_with_NaN[rows_with_NaN['ga:productDetailViews']>=2].shape[0]+rows_with_NaN[rows_with_NaN['ga:productAddsToCart']>=2].shape[0]
rows_with_NaN_agg_obs

521

In [27]:
join_df_agg_obs = join_df[join_df['ga:productListViews']>=2].shape[0] + join_df[join_df['ga:productListClicks']>=2].shape[0]+join_df[join_df['ga:productDetailViews']>=2].shape[0]+join_df[join_df['ga:productAddsToCart']>=2].shape[0]
join_df_agg_obs

10107

521 filas de las 1100 que tienen NaNs tienes valores superiores a 1 en alguna de las columnas que hacen referencia a hits.

De las 10.107 filas de join_df que tienen valores superiores a 1 en algunas de las columnas de hits, 521 tienen NaNs.

En definitiva, en estos casos lo que tenemos son varios usuarios, el mismo día, a la misma hora viendo la misma URL, ya sea en páginas tipo lista o páginas especificas de producto. Probemos a incluir una nueva dimensión común que nos ayude a diferenciar en mayor medida a los usuarios y, por tanto, a desagregar mejor los datos.

GA no cuenta con una dimension que nos permita identificar a los usuarios únicos pero exploremos que otras dimensiones descriptivas pueden resultar en productListViews y productDetailViews con valores más próximos a 0 u 1. Empezaremos haciendolo con las columnas con las que ya contamos en join_df:

In [28]:
join_df.describe()

,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart
count,226195.000000,226195.000000,226195.000000,226195.000000
mean,0.952139,0.008793,0.093464,0.010787
std,0.325734,0.095837,0.376004,0.108965
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000,0.000000
max,9.000000,6.000000,13.000000,6.000000


In [29]:
join_df['ga:sessionCount'] = pd.to_numeric(join_df['ga:sessionCount'])
join_df['ga:daysSinceLastSession'] = pd.to_numeric(join_df['ga:daysSinceLastSession'])
join_df['ga:pageDepth'] = pd.to_numeric(join_df['ga:pageDepth'])
join_df.describe()

,ga:sessionCount,ga:daysSinceLastSession,ga:pageDepth,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart
count,225671.000000,225671.000000,225671.000000,226195.000000,226195.000000,226195.000000,226195.000000
mean,2.984553,4.072375,12.141285,0.952139,0.008793,0.093464,0.010787
std,6.584178,16.165338,14.889992,0.325734,0.095837,0.376004,0.108965
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,3.000000,1.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,7.000000,1.000000,0.000000,0.000000,0.000000
75%,2.000000,0.000000,16.000000,1.000000,0.000000,0.000000,0.000000
max,131.000000,179.000000,110.000000,9.000000,6.000000,13.000000,6.000000


In [262]:
groupby = join_df.groupby('ga:sessionCount')[['ga:productListViews','ga:productDetailViews']].describe()
print("Mean/Std for productListViews:")
print(groupby['ga:productListViews']['mean'].mean(), groupby['ga:productListViews']['std'].mean())
print("Mean/Std for productDetailViews:")
print(groupby['ga:productDetailViews']['mean'].mean(), groupby['ga:productDetailViews']['std'].mean())
groupby

Mean/Std for productListViews:
0.7448679092499394 0.25400148420040813
Mean/Std for productDetailViews:
0.33123100986891696 0.24740305919613206


ga:productListViews                                          \
                              count      mean       std  min  25%  50%  75%   
ga:sessionCount                                                               
1.0                        137471.0  0.949582  0.332952  0.0  1.0  1.0  1.0   
2.0                         32776.0  0.942916  0.320859  0.0  1.0  1.0  1.0   
3.0                         17212.0  0.956658  0.306389  0.0  1.0  1.0  1.0   
4.0                         10177.0  0.974550  0.327945  0.0  1.0  1.0  1.0   
5.0                          5078.0  0.951753  0.284585  0.0  1.0  1.0  1.0   
...                             ...       ...       ...  ...  ...  ...  ...   
127.0                           1.0  0.000000       NaN  0.0  0.0  0.0  0.0   
128.0                           1.0  0.000000       NaN  0.0  0.0  0.0  0.0   
129.0                           1.0  0.000000       NaN  0.0  0.0  0.0  0.0   
130.0                           1.0  0.000000       NaN  0.0  0.0  0.0  0.0   
131.0                          82.0  0.987805  0.110432  0.0  1.0  1.0  1.0   

                     ga:productDetailViews                                     \
                 max                 count      mean       std  min  25%  50%   
ga:sessionCount                                                                 
1.0              5.0              137471.0  0.106255  0.415728  0.0  0.0  0.0   
2.0              9.0               32776.0  0.082530  0.315274  0.0  0.0  0.0   
3.0              3.0               17212.0  0.069835  0.293249  0.0  0.0  0.0   
4.0              3.0               10177.0  0.060136  0.264738  0.0  0.0  0.0   
5.0              3.0                5078.0  0.068728  0.289333  0.0  0.0  0.0   
...              ...                   ...       ...       ...  ...  ...  ...   
127.0            0.0                   1.0  1.000000       NaN  1.0  1.0  1.0   
128.0            0.0                   1.0  1.000000       NaN  1.0  1.0  1.0   
129.0            0.0                   1.0  1.000000       NaN  1.0  1.0  1.0   
130.0            0.0                   1.0  1.000000       NaN  1.0  1.0  1.0   
131.0            1.0                  82.0  0.012195  0.110432  0.0  0.0  0.0   

                            
                 75%   max  
ga:sessionCount             
1.0              0.0  13.0  
2.0              0.0   5.0  
3.0              0.0   5.0  
4.0              0.0   4.0  
5.0              0.0   5.0  
...              ...   ...  
127.0            1.0   1.0  
128.0            1.0   1.0  
129.0            1.0   1.0  
130.0            1.0   1.0  
131.0            0.0   1.0  

[81 rows x 16 columns]

In [263]:
groupby = join_df.groupby('ga:daysSinceLastSession')[['ga:productListViews','ga:productDetailViews']].describe()
print("Mean/Std for productListViews:")
print(groupby['ga:productListViews']['mean'].mean(), groupby['ga:productListViews']['std'].mean())
print("Mean/Std for productDetailViews:")
print(groupby['ga:productDetailViews']['mean'].mean(), groupby['ga:productDetailViews']['std'].mean())
groupby

Mean/Std for productListViews:
0.7500665291853021 0.31527526701715314
Mean/Std for productDetailViews:
0.40930545316772227 0.43909420580479824


ga:productListViews                                \
                                      count      mean       std  min  25%   
ga:daysSinceLastSession                                                     
0.0                                185694.0  0.950251  0.326905  0.0  1.0   
1.0                                  6204.0  0.958414  0.295397  0.0  1.0   
2.0                                  3137.0  0.952184  0.282813  0.0  1.0   
3.0                                  4379.0  0.968258  0.270605  0.0  1.0   
4.0                                  1931.0  0.957535  0.334941  0.0  1.0   
...                                     ...       ...       ...  ...  ...   
162.0                                   6.0  0.833333  0.408248  0.0  1.0   
163.0                                   1.0  0.000000       NaN  0.0  0.0   
177.0                                  15.0  0.933333  0.258199  0.0  1.0   
178.0                                  12.0  0.833333  0.389249  0.0  1.0   
179.0                                  86.0  1.011628  0.187503  0.0  1.0   

                                       ga:productDetailViews            \
                         50%  75%  max                 count      mean   
ga:daysSinceLastSession                                                  
0.0                      1.0  1.0  9.0              185694.0  0.096336   
1.0                      1.0  1.0  2.0                6204.0  0.064475   
2.0                      1.0  1.0  2.0                3137.0  0.061205   
3.0                      1.0  1.0  3.0                4379.0  0.052752   
4.0                      1.0  1.0  3.0                1931.0  0.070948   
...                      ...  ...  ...                   ...       ...   
162.0                    1.0  1.0  1.0                   6.0  0.333333   
163.0                    0.0  0.0  0.0                   1.0  2.000000   
177.0                    1.0  1.0  1.0                  15.0  0.066667   
178.0                    1.0  1.0  1.0                  12.0  0.166667   
179.0                    1.0  1.0  2.0                  86.0  0.011628   

                                                             
                              std  min  25%  50%  75%   max  
ga:daysSinceLastSession                                      
0.0                      0.384477  0.0  0.0  0.0  0.0  13.0  
1.0                      0.258410  0.0  0.0  0.0  0.0   3.0  
2.0                      0.248881  0.0  0.0  0.0  0.0   2.0  
3.0                      0.244078  0.0  0.0  0.0  0.0   3.0  
4.0                      0.279971  0.0  0.0  0.0  0.0   3.0  
...                           ...  ...  ...  ...  ...   ...  
162.0                    0.816497  0.0  0.0  0.0  0.0   2.0  
163.0                         NaN  2.0  2.0  2.0  2.0   2.0  
177.0                    0.258199  0.0  0.0  0.0  0.0   1.0  
178.0                    0.389249  0.0  0.0  0.0  0.0   1.0  
179.0                    0.107833  0.0  0.0  0.0  0.0   1.0  

[139 rows x 16 columns]

In [264]:
groupby = join_df.groupby('ga:pageDepth')[['ga:productListViews','ga:productDetailViews']].describe()
print("Mean/Std for productListViews:")
print(groupby['ga:productListViews']['mean'].mean(), groupby['ga:productListViews']['std'].mean())
print("Mean/Std for productDetailViews:")
print(groupby['ga:productDetailViews']['mean'].mean(), groupby['ga:productDetailViews']['std'].mean())
groupby

Mean/Std for productListViews:
0.9832608403290933 0.29770242230081834
Mean/Std for productDetailViews:
0.05190611532824781 0.23530256134589714


ga:productListViews                                               \
                           count      mean       std  min  25%  50%  75%  max   
ga:pageDepth                                                                    
0.0                        859.0  0.948778  0.220579  0.0  1.0  1.0  1.0  1.0   
1.0                      17442.0  0.825364  0.391708  0.0  1.0  1.0  1.0  3.0   
2.0                      22211.0  0.912026  0.367408  0.0  1.0  1.0  1.0  4.0   
3.0                      24970.0  0.908811  0.404793  0.0  1.0  1.0  1.0  3.0   
4.0                      16596.0  0.944987  0.324808  0.0  1.0  1.0  1.0  3.0   
5.0                      16390.0  0.967053  0.285963  0.0  1.0  1.0  1.0  5.0   
6.0                      11055.0  0.967888  0.284323  0.0  1.0  1.0  1.0  3.0   
7.0                       8314.0  0.964518  0.280599  0.0  1.0  1.0  1.0  2.0   
8.0                       8915.0  0.982614  0.274629  0.0  1.0  1.0  1.0  4.0   
9.0                       7829.0  0.972155  0.264085  0.0  1.0  1.0  1.0  5.0   
10.0                      6220.0  0.971704  0.303546  0.0  1.0  1.0  1.0  3.0   
11.0                      6225.0  0.989558  0.272544  0.0  1.0  1.0  1.0  5.0   
12.0                      6056.0  0.974736  0.270527  0.0  1.0  1.0  1.0  3.0   
13.0                      5786.0  0.989803  0.288170  0.0  1.0  1.0  1.0  3.0   
14.0                      5532.0  0.995481  0.285517  0.0  1.0  1.0  1.0  4.0   
15.0                      4428.0  0.990967  0.266171  0.0  1.0  1.0  1.0  3.0   
16.0                      4204.0  0.974072  0.229915  0.0  1.0  1.0  1.0  2.0   
17.0                      4153.0  0.977847  0.278467  0.0  1.0  1.0  1.0  3.0   
18.0                      4983.0  0.990367  0.239403  0.0  1.0  1.0  1.0  3.0   
19.0                      3365.0  0.992273  0.277901  0.0  1.0  1.0  1.0  2.0   
20.0                      2487.0  0.976277  0.248568  0.0  1.0  1.0  1.0  3.0   
21.0                      3012.0  0.984728  0.254679  0.0  1.0  1.0  1.0  3.0   
22.0                      3097.0  0.989345  0.266962  0.0  1.0  1.0  1.0  2.0   
23.0                      1138.0  1.000000  0.461347  0.0  1.0  1.0  1.0  4.0   
24.0                      1619.0  0.951822  0.261026  0.0  1.0  1.0  1.0  2.0   
25.0                      2111.0  1.018475  0.340253  0.0  1.0  1.0  1.0  3.0   
26.0                      1993.0  0.975916  0.273351  0.0  1.0  1.0  1.0  4.0   
27.0                      1647.0  1.020036  0.450691  0.0  1.0  1.0  1.0  3.0   
28.0                      1320.0  0.971970  0.291350  0.0  1.0  1.0  1.0  3.0   
29.0                      1846.0  1.000542  0.286081  0.0  1.0  1.0  1.0  3.0   
30.0                       235.0  0.965957  0.181726  0.0  1.0  1.0  1.0  1.0   
31.0                      1151.0  1.017376  0.269706  0.0  1.0  1.0  1.0  3.0   
32.0                       631.0  0.968304  0.312100  0.0  1.0  1.0  1.0  2.0   
33.0                      1361.0  0.970610  0.228203  0.0  1.0  1.0  1.0  2.0   
34.0                        93.0  0.935484  0.527637  0.0  1.0  1.0  1.0  2.0   
35.0                       475.0  1.067368  0.485755  0.0  1.0  1.0  1.0  4.0   
36.0                       421.0  1.014252  0.323355  0.0  1.0  1.0  1.0  2.0   
37.0                      1247.0  1.009623  0.256355  0.0  1.0  1.0  1.0  4.0   
38.0                       813.0  0.996310  0.207581  0.0  1.0  1.0  1.0  2.0   
39.0                      1432.0  0.986034  0.174793  0.0  1.0  1.0  1.0  2.0   
40.0                       660.0  0.983333  0.194057  0.0  1.0  1.0  1.0  2.0   
41.0                       266.0  1.000000  0.325054  0.0  1.0  1.0  1.0  3.0   
42.0                      1136.0  0.966549  0.219589  0.0  1.0  1.0  1.0  2.0   
43.0                      1172.0  0.975256  0.203056  0.0  1.0  1.0  1.0  2.0   
45.0                       391.0  1.000000  0.237508  0.0  1.0  1.0  1.0  2.0   
46.0                       228.0  0.947368  0.260197  0.0  1.0  1.0  1.0  2.0   
47.0                       445.0  1.051685  

In [265]:
groupby = join_df.groupby('ga:sourceMedium')[['ga:productListViews','ga:productDetailViews']].describe()
print("Mean/Std for productListViews:")
print(groupby['ga:productListViews']['mean'].mean(), groupby['ga:productListViews']['std'].mean())
print("Mean/Std for productDetailViews:")
print(groupby['ga:productDetailViews']['mean'].mean(), groupby['ga:productDetailViews']['std'].mean())
groupby

Mean/Std for productListViews:
0.8540556196579034 0.2730931164363773
Mean/Std for productDetailViews:
0.1732113685621372 0.27661251209528215


ga:productListViews            \
                                                              count      mean   
ga:sourceMedium                                                                 
(direct) / (none)                                           25911.0  0.966346   
IGShopping / Social                                           885.0  0.944633   
abcblogs.abc.es / referral                                    241.0  0.962656   
abiertosparati.es / referral                                  220.0  1.004545   
baidu / organic                                                 1.0  0.000000   
billionhands.com / referral                                    11.0  0.727273   
bing / organic                                                759.0  0.968379   
cantabrialabs.es / referral                                  2039.0  1.029917   
duckduckgo / organic                                          180.0  0.977778   
ecosia.org / organic                                           38.0  0.973684   
es.labo-svr.com / referral                                      1.0  0.000000   
es.search.yahoo.com / referral                                 47.0  0.893617   
facebook.com / referral                                       523.0  1.074570   
goahclinic.com / referral                                      32.0  1.000000   
google / cpc                                                40626.0  0.887855   
google / organic                                            86266.0  0.968806   
idealo.lightning.force.com / referral                          79.0  0.987342   
instagram.com / referral                                     9415.0  1.006798   
l.facebook.com / referral                                     274.0  1.018248   
l.instagram.com / referral                                   7462.0  0.959260   
m.facebook.com / referral                                    2560.0  0.978906   
m.youtube.com / referral                                      118.0  0.669492   
opiniones2.opiniones-verificadas.com / referral               110.0  0.972727   
secure.comnpay.com / referral                                  35.0  0.857143   
sendinblue / email                                          37536.0  0.998854   
uniquepinkcollagen.com / referral                              79.0  0.987342   
vogue.es / referral                                           492.0  0.997967   
www-vogue-es.cdn.ampproject.org / referral                    195.0  0.948718   
yahoo / organic                                                74.0  0.959459   
youtube.com / referral                                       9461.0  0.753409   
zapp.plyzer.ai / referral                                       1.0  0.000000   

                                                                               \
                                                      std  min  25%  50%  75%   
ga:sourceMedium                                                                 
(direct) / (none)                                0.317668  0.0  1.0  1.0  1.0   
IGShopping / Social                              0.316026  0.0  1.0  1.0  1.0   
abcblogs.abc.es / referral                       0.210791  0.0  1.0  1.0  1.0   
abiertosparati.es / referral                     0.067420  1.0  1.0  1.0  1.0   
baidu / organic                                       NaN  0.0  0.0  0.0  0.0   
billionhands.com / referral                      0.467099  0.0  0.5  1.0  1.0   
bing / organic                                   0.339255  0.0  1.0  1.0  1.0   
cantabrialabs.es / referral                      0.362073  0.0  1.0  1.0  1.0   
duckduckgo / organic                             0.147817  0.0  1.0  1.0  1.0   
ecosia.org / organic                             0.366639  0.0  1.0  1.0  1.0   
es.labo-svr.com / referral                            NaN  0.0  0.0  0.0  0.0   
es.search.yahoo.com / referral                   0.311661  0.0  1.0  1.0  1.0   
facebook.com / referral                          0.277136  0.0  1.0  1.0  1.0 

In [266]:
groupby = join_df.groupby('ga:city')[['ga:productListViews','ga:productDetailViews']].describe()
print("Mean/Std for productListViews:")
print(groupby['ga:productListViews']['mean'].mean(), groupby['ga:productListViews']['std'].mean())
print("Mean/Std for productDetailViews:")
print(groupby['ga:productDetailViews']['mean'].mean(), groupby['ga:productDetailViews']['std'].mean())
groupby

Mean/Std for productListViews:
0.6698166720930219 0.30103545457198033
Mean/Std for productDetailViews:
0.5152731755077952 0.4628305483124371


ga:productListViews                                          \
                             count      mean       std  min  25%  50%  75%   
ga:city                                                                      
(not set)                  36375.0  0.949746  0.314079  0.0  1.0  1.0  1.0   
A Coruna                    1864.0  0.943133  0.289346  0.0  1.0  1.0  1.0   
Acapulco                      16.0  0.937500  0.250000  0.0  1.0  1.0  1.0   
Aguilas                       86.0  0.860465  0.464322  0.0  1.0  1.0  1.0   
Albacete                    1172.0  0.936007  0.289587  0.0  1.0  1.0  1.0   
...                            ...       ...       ...  ...  ...  ...  ...   
Zagreb                         1.0  0.000000       NaN  0.0  0.0  0.0  0.0   
Zamora                        57.0  1.000000  0.755929  0.0  0.0  1.0  2.0   
Zapopan                        1.0  0.000000       NaN  0.0  0.0  0.0  0.0   
Zaragoza                    1948.0  0.925565  0.337828  0.0  1.0  1.0  1.0   
la Vall d'Uixo                 6.0  0.000000  0.000000  0.0  0.0  0.0  0.0   

                    ga:productDetailViews                                     \
                max                 count      mean       std  min  25%  50%   
ga:city                                                                        
(not set)       5.0               36375.0  0.092536  0.373878  0.0  0.0  0.0   
A Coruna        3.0                1864.0  0.091202  0.371051  0.0  0.0  0.0   
Acapulco        1.0                  16.0  0.062500  0.250000  0.0  0.0  0.0   
Aguilas         2.0                  86.0  0.197674  0.480680  0.0  0.0  0.0   
Albacete        2.0                1172.0  0.089590  0.359800  0.0  0.0  0.0   
...             ...                   ...       ...       ...  ...  ...  ...   
Zagreb          0.0                   1.0  1.000000       NaN  1.0  1.0  1.0   
Zamora          2.0                  57.0  0.403509  0.728493  0.0  0.0  0.0   
Zapopan         0.0                   1.0  1.000000       NaN  1.0  1.0  1.0   
Zaragoza        3.0                1948.0  0.120637  0.418284  0.0  0.0  0.0   
la Vall d'Uixo  0.0                   6.0  1.666667  1.032796  1.0  1.0  1.0   

                          
                75%  max  
ga:city                   
(not set)       0.0  5.0  
A Coruna        0.0  3.0  
Acapulco        0.0  1.0  
Aguilas         0.0  2.0  
Albacete        0.0  3.0  
...             ...  ...  
Zagreb          1.0  1.0  
Zamora          1.0  3.0  
Zapopan         1.0  1.0  
Zaragoza        0.0  3.0  
la Vall d'Uixo  2.5  3.0  

[333 rows x 16 columns]

In [267]:
groupby = join_df.groupby('ga:deviceCategory')[['ga:productListViews','ga:productDetailViews']].describe()
print("Mean/Std for productListViews:")
print(groupby['ga:productListViews']['mean'].mean(), groupby['ga:productListViews']['std'].mean())
print("Mean/Std for productDetailViews:")
print(groupby['ga:productDetailViews']['mean'].mean(), groupby['ga:productDetailViews']['std'].mean())
groupby

Mean/Std for productListViews:
0.9563317791649247 0.31465295450052194
Mean/Std for productDetailViews:
0.08318960797130312 0.3418391707312039


ga:productListViews                                          \
                                count      mean       std  min  25%  50%  75%   
ga:deviceCategory                                                               
desktop                       56401.0  0.968653  0.313654  0.0  1.0  1.0  1.0   
mobile                       158307.0  0.945751  0.327334  0.0  1.0  1.0  1.0   
tablet                        10901.0  0.954591  0.302970  0.0  1.0  1.0  1.0   

                       ga:productDetailViews                                \
                   max                 count      mean       std  min  25%   
ga:deviceCategory                                                            
desktop            9.0               56401.0  0.069219  0.304801  0.0  0.0   
mobile             5.0              158307.0  0.101183  0.392156  0.0  0.0   
tablet             2.0               10901.0  0.079167  0.328561  0.0  0.0   

                                  
                   50%  75%  max  
ga:deviceCategory                 
desktop            0.0  0.0  5.0  
mobile             0.0  0.0  9.0  
tablet             0.0  0.0  4.0

In [268]:
groupby = join_df.groupby('ga:operatingSystem')[['ga:productListViews','ga:productDetailViews']].describe()
print("Mean/Std for productListViews:")
print(groupby['ga:productListViews']['mean'].mean(), groupby['ga:productListViews']['std'].mean())
print("Mean/Std for productDetailViews:")
print(groupby['ga:productDetailViews']['mean'].mean(), groupby['ga:productDetailViews']['std'].mean())
groupby

Mean/Std for productListViews:
0.9431232746713127 0.38627599198074214
Mean/Std for productDetailViews:
0.12732142544540243 0.3931469394712173


ga:productListViews                                     \
                                 count      mean       std  min  25%  50%   
ga:operatingSystem                                                          
(not set)                         10.0  0.800000  0.421637  0.0  1.0  1.0   
Android                       103068.0  0.948452  0.326172  0.0  1.0  1.0   
BlackBerry                        40.0  0.975000  0.422902  0.0  1.0  1.0   
Chrome OS                        509.0  0.944990  0.417293  0.0  1.0  1.0   
Linux                            296.0  0.996622  0.561465  0.0  1.0  1.0   
Macintosh                      21433.0  0.967667  0.295433  0.0  1.0  1.0   
Windows                        34150.0  0.969107  0.320345  0.0  1.0  1.0   
iOS                            66103.0  0.943149  0.324961  0.0  1.0  1.0   

                             ga:productDetailViews                           \
                    75%  max                 count      mean       std  min   
ga:operatingSystem                                                            
(not set)           1.0  1.0                  10.0  0.200000  0.421637  0.0   
Android             1.0  5.0              103068.0  0.099721  0.391794  0.0   
BlackBerry          1.0  2.0                  40.0  0.150000  0.426675  0.0   
Chrome OS           1.0  2.0                 509.0  0.094303  0.312072  0.0   
Linux               1.0  2.0                 296.0  0.239865  0.610556  0.0   
Macintosh           1.0  3.0               21433.0  0.064247  0.289364  0.0   
Windows             1.0  9.0               34150.0  0.070835  0.310280  0.0   
iOS                 1.0  5.0               66103.0  0.099602  0.382798  0.0   

                                        
                    25%  50%  75%  max  
ga:operatingSystem                      
(not set)           0.0  0.0  0.0  1.0  
Android             0.0  0.0  0.0  6.0  
BlackBerry          0.0  0.0  0.0  2.0  
Chrome OS           0.0  0.0  0.0  2.0  
Linux               0.0  0.0  0.0  3.0  
Macintosh           0.0  0.0  0.0  4.0  
Windows             0.0  0.0  0.0  5.0  
iOS                 0.0  0.0  0.0  9.0

In [269]:
groupby = join_df.groupby('ga:campaign')[['ga:productListViews','ga:productDetailViews']].describe()
print("Mean/Std for productListViews:")
print(groupby['ga:productListViews']['mean'].mean(), groupby['ga:productListViews']['std'].mean())
print("Mean/Std for productDetailViews:")
print(groupby['ga:productDetailViews']['mean'].mean(), groupby['ga:productDetailViews']['std'].mean())
groupby

Mean/Std for productListViews:
0.8655140955825558 0.24538550623977626
Mean/Std for productDetailViews:
0.1647301504406157 0.2646465502788423


ga:productListViews                           \
                                            count      mean       std  min   
ga:campaign                                                                  
(not set)                                147391.0  0.958118  0.320527  0.0   
BF_Domingo__22NOV_BlackFriday               122.0  1.000000  0.000000  1.0   
BF_ltimo_aviso_Viernes_20NOV                  8.0  1.000000  0.000000  1.0   
BF_sabado__28NOV_BlackFriday                234.0  0.974359  0.205568  0.0   
Champions shopping                        21250.0  0.893365  0.354234  0.0   
Champions shopping bto                       28.0  0.785714  0.417855  0.0   
Navidad_2020                                907.0  1.016538  0.358978  0.0   
Rebajas_enero_2020_General                23533.0  1.000340  0.244782  0.0   
Rebajas_enero_2020_VIP                    12747.0  0.994116  0.260140  0.0   
Resto Oportunidades                       12318.0  0.871570  0.410033  0.0   
Uefa Shopping                              7068.0  0.892049  0.372508  0.0   
Uefa shopping bto                             3.0  0.000000  0.000000  0.0   

                                                  ga:productDetailViews  \
                               25%  50%  75%  max                 count   
ga:campaign                                                               
(not set)                      1.0  1.0  1.0  9.0              147391.0   
BF_Domingo__22NOV_BlackFriday  1.0  1.0  1.0  1.0                 122.0   
BF_ltimo_aviso_Viernes_20NOV   1.0  1.0  1.0  1.0                   8.0   
BF_sabado__28NOV_BlackFriday   1.0  1.0  1.0  2.0                 234.0   
Champions shopping             1.0  1.0  1.0  5.0               21250.0   
Champions shopping bto         1.0  1.0  1.0  1.0                  28.0   
Navidad_2020                   1.0  1.0  1.0  3.0                 907.0   
Rebajas_enero_2020_General     1.0  1.0  1.0  4.0               23533.0   
Rebajas_enero_2020_VIP         1.0  1.0  1.0  3.0               12747.0   
Resto Oportunidades            1.0  1.0  1.0  4.0               12318.0   
Uefa Shopping                  1.0  1.0  1.0  3.0                7068.0   
Uefa shopping bto              0.0  0.0  0.0  0.0                   3.0   

                                                                            
                                   mean       std  min  25%  50%  75%  max  
ga:campaign                                                                 
(not set)                      0.084924  0.350685  0.0  0.0  0.0  0.0  5.0  
BF_Domingo__22NOV_BlackFriday  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  
BF_ltimo_aviso_Viernes_20NOV   0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  
BF_sabado__28NOV_BlackFriday   0.029915  0.194237  0.0  0.0  0.0  0.0  2.0  
Champions shopping             0.157882  0.487179  0.0  0.0  0.0  0.0  9.0  
Champions shopping bto         0.214286  0.498675  0.0  0.0  0.0  0.0  2.0  
Navidad_2020                   0.047409  0.222770  0.0  0.0  0.0  0.0  2.0  
Rebajas_enero_2020_General     0.023116  0.157187  0.0  0.0  0.0  0.0  4.0  
Rebajas_enero_2020_VIP         0.029732  0.177975  0.0  0.0  0.0  0.0  3.0  
Resto Oportunidades            0.224387  0.594814  0.0  0.0  0.0  0.0  5.0  
Uefa Shopping                  0.165110  0.492237  0.0  0.0  0.0  0.0  4.0  
Uefa shopping bto              1.000000  0.000000  1.0  1.0  1.0  1.0  1.0

In [57]:
def views_per_dim(df,dims):
    
    metrics = []
    
    for dim in dims:
        groupby=df.groupby(dim)[['ga:productListViews','ga:productDetailViews']].describe()
        average_metrics_per_dim = [dim, groupby['ga:productListViews']['mean'].mean(),groupby['ga:productListViews']['std'].mean(), groupby['ga:productDetailViews']['mean'].mean(), groupby['ga:productDetailViews']['std'].mean()]
        metrics.append(average_metrics_per_dim)
        
    views_per_dim =pd.DataFrame(metrics)
    
    return views_per_dim
    
dims = ['ga:sessionCount', 'ga:daysSinceLastSession', 'ga:pageDepth', 'ga:sourceMedium', 'ga:city', 'ga:deviceCategory', 'ga:operatingSystem', 'ga:campaign']
views_per_dim = views_per_dim(join_df, dims)
views_per_dim.columns = ["Dimension","Listviews_mean", "Listviews_std", "Detailviews_mean", "Detailviews_std"]
views_per_dim

,Dimension,Listviews_mean,Listviews_std,Detailviews_mean,Detailviews_std
0,ga:sessionCount,0.744868,0.254001,0.331231,0.247403
1,ga:daysSinceLastSession,0.750067,0.315275,0.409305,0.439094
2,ga:pageDepth,0.983261,0.297702,0.051906,0.235303
3,ga:sourceMedium,0.854056,0.273093,0.173211,0.276613
4,ga:city,0.669817,0.301035,0.515273,0.462831
5,ga:deviceCategory,0.956332,0.314653,0.083190,0.341839
6,ga:operatingSystem,0.943123,0.386276,0.127321,0.393147
7,ga:campaign,0.865514,0.245386,0.164730,0.264647


Observamos que la dimensión 'city' y 'sessionCount' son las que más tienden a adoptar valores entre 0 y 1 para productListViews. Esto resulta lógico ya que, al adoptar mayor número de valores (también captan más información de los usuarios y sus diferencias), se diluyen los casos con ListViews con valores superiores a 1. 

En cambio, 'pageDepth' y 'deviceCategory' tienden a valores más elevados por su poca explicabilidad.

En el caso de pageDetailViews, parecen invertirse los papeles aunque los valores resultan mucho menos elevados. 

En cualquier caso, haré dos pruebas para ver si podemos reducir el número de filas con NaNs de manera significativa:
- Prueba 1: Añadimos como variable común 'city'. Mantenemos second_request como está y relanzamos first_request con 'city' en lugar de 'pageDepth'.

- Prueba 2: Añadimos como variable común 'countSession'. Mantenemos first_request como está y relanzamos second_request con 'countSession' en lugar de 'deviceCategory'.

#### PRUEBA 1

Añadimos como variable común 'city'. Mantenemos second_request como está y relanzamos first_request con 'city' en lugar de 'pageDepth'

In [47]:
# FIRST REQUEST without pageDepth with city:

pagetokens = [None] + [str(x) for x in range(0,1100000,100000)][1:]
list_ = [] # empty list for row data
sampled_data = [] #empty list for sampling results
next_page_token_list = [] #empty list for next_page_tokens in each iteration
full_data_loaded = "All the data were loaded for this period and request" #Control message

for pagetoken in pagetokens:

    #REPORT REQUEST:
    response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'2021-01-01','endDate':'2021-01-07'}],
        'metrics':
        [
            {'expression':'ga:productListViews'},
            {'expression':'ga:productListClicks'},
            {'expression':'ga:productDetailViews'},
            {'expression':'ga:productAddsToCart'},
        ],
        'dimensions':
        [
            {'name':'ga:productName'},
            {'name':'ga:pagePath'},
            {'name':'ga:dateHourMinute'},
            {'name':'ga:sessionCount'},
            {'name':'ga:daysSinceLastSession'},
            {'name':'ga:sourceMedium'},
            {'name':'ga:city'},
        ],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        #'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        "pageToken": pagetoken, #extract data from pagetoken index
        'pageSize':100000 #if we do not specify we will get 1000 rows
    }]
}).execute()

    #PARSING THE REPORT DATA:
    
    for report in response.get('reports',[]):
        
        columnHeader = report.get('columnHeader',{})
        dimensionHeaders = columnHeader.get('dimensions',[])
        metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])
        
        rows = report.get('data',{}).get('rows',[])
        for row in rows:
            dict_={}
            dimensions = row.get('dimensions',[])
            dateRangeValues=row.get('metrics',[])
            
            for header, dimension in zip(dimensionHeaders,dimensions):
                dict_[header]=dimension
            
            for i,values in enumerate (dateRangeValues):
                for metricHeader,value in zip(metricHeaders,values.get('values')):
                    if ',' in value or '.' in value:
                      dict_[metricHeader.get('name')] = float(value)
                    else:
                      dict_[metricHeader.get('name')] = int(value)
                    
            list_.append(dict_)

        # Sampled Data? : 
        sampling = "No sampled data"
        sample_sizes = report.get('data',{}).get('samplesReadCounts',[])
        sample_spaces = report.get('data',{}).get('samplingSpaceSizes',[])
        if sample_sizes and sample_spaces:
            sampling = "Sampled data"
        sampled_data.append(sampling)
        
        # NextPageToken for following queries:    
        next_page_token = report.get('nextPageToken',None)
        next_page_token_list.append(str(next_page_token))
        
    if next_page_token == None: #to stop iterations once all the data are loaded
        break
    if int(next_page_token) > 1000000: #to warn about remaining pages
        full_data_loaded = "Remaining data to request. Please update pagetokens variable."

#DATAFRAME WITH DATA:
import pandas as pd
first_request_withCity_withoutPageDepth= pd.DataFrame(list_)
print((6*"*")+"Results of our request"+(6*"*"))
print("Shape of the final dataframe: " + str(first_request_withCity_withoutPageDepth.shape))
print("Number of requests or iterations: " + str(len(next_page_token_list)))
print("Sampling for each iteration: " + str(sampled_data))
print("Next page token for each iteration: " + str(next_page_token_list))
print("Were all data exported? " + full_data_loaded)
print("Five first rows of our dataframe:")
first_request_withCity_withoutPageDepth.head()

******Results of our request******
Shape of the final dataframe: (215649, 11)
Number of requests or iterations: 3
Sampling for each iteration: ['No sampled data', 'No sampled data', 'No sampled data']
Next page token for each iteration: ['100000', '200000', 'None']
Were all data exported? All the data were loaded for this period and request
Five first rows of our dataframe:


,ga:productName,ga:pagePath,ga:dateHourMinute,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:city,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart
0,(not set),/es/2-inicio,202101072258,2,13,youtube.com / referral,Madrid,1,0,0,0
1,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1041-neostrata-basis-redox-crema-antiarrug...,202101031955,4,1,google / cpc,(not set),1,0,0,0
2,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1086-neostrata-skin-active-cellular-restor...,202101050039,3,0,cantabrialabs.es / referral,Valladolid,1,0,0,0
3,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1086-neostrata-skin-active-cellular-restor...,202101050059,3,0,cantabrialabs.es / referral,Valladolid,1,0,0,0
4,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/4567-medik8-super-c30-intense-30-ml-818625...,202101062119,1,0,google / cpc,Alcorcon,1,0,0,0


In [61]:
first_request.shape

(215561, 11)

In [33]:
second_request.shape

(215589, 11)

In [48]:
first_request_withCity_withoutPageDepth.shape

(215649, 11)

In [49]:
second_request.shape[0] - first_request_withCity_withoutPageDepth.shape[0]

-60

In [50]:
join_df2 = first_request_withCity_withoutPageDepth.merge(second_request, how='outer')
join_df2.shape

(216874, 14)

In [51]:
print(join_df.shape)

(226195, 15)


In [52]:
is_NaN = join_df2.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = join_df2[row_has_NaN]
rows_with_NaN.shape

(856, 14)

Hemos pasado de tener 1110 filas con NaNs a 856. Hemos conseguido reducir 254 filas. 

In [54]:
1110/216874

0.005118179219270175

In [55]:
856/216874

0.003946992262788531

#### PRUEBA 2

Añadimos como variable común 'countSession'. Mantenemos first_request como está y relanzamos second_request con 'countSession' en lugar de 'deviceCategory'.

In [58]:
#SECOND REQUEST:

pagetokens = [None] + [str(x) for x in range(0,1100000,100000)][1:]
list_ = [] # empty list for row data
sampled_data = [] #empty list for sampling results
next_page_token_list = [] #empty list for next_page_tokens in each iteration
full_data_loaded = "All the data were loaded for this period and request" #Control message

for pagetoken in pagetokens:

    #REPORT REQUEST:
    response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'2021-01-01','endDate':'2021-01-07'}],
        'metrics':
        [
            {'expression':'ga:productListViews'},
            {'expression':'ga:productListClicks'},
            {'expression':'ga:productDetailViews'},
            {'expression':'ga:productAddsToCart'},
        ],
        'dimensions':
        [
            {'name':'ga:productName'},
            {'name':'ga:pagePath'},
            {'name':'ga:dateHourMinute'},
            {'name':'ga:sessionCount'},
            {'name':'ga:city'},
            {'name':'ga:operatingSystem'},
            {'name':'ga:campaign'}
        ],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        #'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        "pageToken": pagetoken, #extract data from pagetoken index
        'pageSize':100000 #if we do not specify we will get 1000 rows
    }]
}).execute()

    #PARSING THE REPORT DATA:
    
    for report in response.get('reports',[]):
        
        columnHeader = report.get('columnHeader',{})
        dimensionHeaders = columnHeader.get('dimensions',[])
        metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])
        
        rows = report.get('data',{}).get('rows',[])
        for row in rows:
            dict_={}
            dimensions = row.get('dimensions',[])
            dateRangeValues=row.get('metrics',[])
            
            for header, dimension in zip(dimensionHeaders,dimensions):
                dict_[header]=dimension
            
            for i,values in enumerate (dateRangeValues):
                for metricHeader,value in zip(metricHeaders,values.get('values')):
                    if ',' in value or '.' in value:
                      dict_[metricHeader.get('name')] = float(value)
                    else:
                      dict_[metricHeader.get('name')] = int(value)
                    
            list_.append(dict_)

        # Sampled Data? : 
        sampling = "No sampled data"
        sample_sizes = report.get('data',{}).get('samplesReadCounts',[])
        sample_spaces = report.get('data',{}).get('samplingSpaceSizes',[])
        if sample_sizes and sample_spaces:
            sampling = "Sampled data"
        sampled_data.append(sampling)
        
        # NextPageToken for following queries:    
        next_page_token = report.get('nextPageToken',None)
        next_page_token_list.append(str(next_page_token))
        
    if next_page_token == None: #to stop iterations once all the data are loaded
        break
    if int(next_page_token) > 1000000: #to warn about remaining pages
        full_data_loaded = "Remaining data to request. Please update pagetokens variable."

#DATAFRAME WITH DATA:
import pandas as pd
second_request_withoutDevCateg_withSessCount= pd.DataFrame(list_)
print((6*"*")+"Results of our request"+(6*"*"))
print("Shape of the final dataframe: " + str(second_request_withoutDevCateg_withSessCount.shape))
print("Number of requests or iterations: " + str(len(next_page_token_list)))
print("Sampling for each iteration: " + str(sampled_data))
print("Next page token for each iteration: " + str(next_page_token_list))
print("Were all data exported? " + full_data_loaded)
print("Five first rows of our dataframe:")
second_request_withoutDevCateg_withSessCount.head()

******Results of our request******
Shape of the final dataframe: (215689, 11)
Number of requests or iterations: 3
Sampling for each iteration: ['No sampled data', 'No sampled data', 'No sampled data']
Next page token for each iteration: ['100000', '200000', 'None']
Were all data exported? All the data were loaded for this period and request
Five first rows of our dataframe:


,ga:productName,ga:pagePath,ga:dateHourMinute,ga:sessionCount,ga:city,ga:operatingSystem,ga:campaign,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart
0,(not set),/es/2-inicio,202101072258,2,Madrid,Android,(not set),1,0,0,0
1,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1041-neostrata-basis-redox-crema-antiarrug...,202101031955,4,(not set),Android,Resto Oportunidades,1,0,0,0
2,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1086-neostrata-skin-active-cellular-restor...,202101050039,3,Valladolid,iOS,(not set),1,0,0,0
3,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/1086-neostrata-skin-active-cellular-restor...,202101050059,3,Valladolid,iOS,(not set),1,0,0,0
4,▷ ▷HELIOCARE 360 Water Gel SPF 50+,/es/4567-medik8-super-c30-intense-30-ml-818625...,202101062119,1,Alcorcon,Android,Resto Oportunidades,1,0,0,0


In [59]:
second_request.shape

(215589, 11)

In [60]:
second_request_withoutDevCateg_withSessCount.shape

(215689, 11)

In [62]:
first_request.shape

(215561, 11)

In [63]:
first_request.shape[0] - second_request_withoutDevCateg_withSessCount.shape[0]

-128

In [64]:
join_df3 = first_request.merge(second_request_withoutDevCateg_withSessCount, how='outer')
join_df3.shape

(220565, 14)

In [65]:
print(join_df.shape)
print(join_df2.shape)

(226195, 15)
(216874, 14)


In [69]:
is_NaN = join_df3.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = join_df3[row_has_NaN]
rows_with_NaN.shape

(865, 14)

In [70]:
rows_with_NaN.sample(50)

,ga:productName,ga:pagePath,ga:dateHourMinute,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:pageDepth,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart,ga:city,ga:operatingSystem,ga:campaign
102357,La Roche Posay Desodorante de bola sin aluminio,/es/module/iqitsearch/searchiqit,202101012126,1,0,google / organic,3,1,0,0,0,NaN,NaN,NaN
55308,Envío Gratuito|Skinceuticals Serum10 TTO Dual ...,/es/serum/6252-skinceuticals-serum-10-30-ml-06...,202101072142,1,0,(direct) / (none),3,0,0,2,0,NaN,NaN,NaN
220321,Loción facial exfoliante 5 AZELAIC-S Gema Herr...,/es/exfoliantes/4393-cristina-mitre-gh5azelaic...,202101032027,1,NaN,NaN,NaN,0,0,1,0,Barcelona,Windows,(not set)
220242,Gema Herrerias 20 AHA peeling químico con vita...,/es/google-shopping/6751-gema-herrerias-crema-...,202101022330,1,NaN,NaN,NaN,1,0,0,0,(not set),iOS,Champions shopping
220156,Cerave loc hidra rostro piel normal 52ml,/es/google-shopping/6751-gema-herrerias-crema-...,202101022329,1,NaN,NaN,NaN,1,0,0,0,Seville,iOS,Champions shopping
220340,Loción facial exfoliante 5 AZELAIC-S Gema Herr...,/es/exfoliantes/4393-cristina-mitre-gh5azelaic...,202101071139,1,NaN,NaN,NaN,0,0,1,1,(not set),iOS,(not set)
181344,Skinceuticals Phloretin Serum | El mejor antio...,/es/,202101010951,1,0,(direct) / (none),4,1,0,0,0,NaN,NaN,NaN
220512,Skinperfection by BLUEVERT Contorno de Ojos 15ml,/es/contorno-de-ojos-para-bolsas/6339-skinperf...,202101031346,1,NaN,NaN,NaN,0,0,2,1,Barcelona,Android,Champions shopping
8263,5punto5 AntiOX Hidratante,/es/,202101010951,1,0,google / organic,5,1,0,0,0,NaN,NaN,NaN
220433,Neostrata Citriate Home Peeling Casero,/es/903-top-ventas?page=2,202101071747,1,NaN,NaN,NaN,2,0,0,0,Madrid,iOS,(not set)


Hemos pasado de tener 1110 filas con NaNs a 865. Hemos conseguido reducir 245 filas. 

#### CONCLUSIONES

Llegados a este punto tenemos 4 opciones:
- OPCION 1: Quedarnos con join_df que si bien tiene mayor número de filas con NaNs también nos permite captar menos número de dimensiones por consulta.
- OPCIÓN 2: Lanzar las consultas con 4 dimensiones comunes (productName, dateHourMinute, pagePath, city) y las 4 metricas relativas a los hits. Ganamos más filas sin NaNs en el merge pero perdemos el feature pageDepth.
- OPCIÓN 3: Lanzar las consultas con 4 dimensiones comunes (productName, dateHourMinute, pagePath, sessionCount) y las 4 metricas relativas a los hits.  Ganamos más filas sin NaNs en el merge pero perdemos el feature deviceCategory.
- OPCIÓN 4: Probamos con otras dimensiones alternativas en el catalogo de Google Analytics que nos ayuden a desagregar en mayor medida los datos.

In [77]:
# porcentaje de filas con NaNS en OPCIÓN 1:

1110 / join_df.shape[0]

0.004907270275647119

In [78]:
# porcentaje de filas con NaNS en OPCIÓN 2:

856 / join_df2.shape[0]

0.003946992262788531

In [79]:
# porcentaje de filas con NaNS en OPCIÓN 3:

865 / join_df3.shape[0]

0.0039217464239566565

Con el fin de lanzar cuanto antes un MVP que nos dé información sobre lo bien o lo mal que funciona el modelo con las dimensiones con las que contamos en la actualidad, descartaremos la opción 4 y, dado que las diferencias entre añadir o no una 4 dimensión común no son significativas, optaremos por la OPCIÓN 1 en un primer momento.

### Max. 50 requests per day

El último reto que nos plantea la API de GA es que solo se pueden realizar 50 requests al día. Quiere decir que si deseamos extraer 3 años de histórico deberemos realizar distintas consultas a lo largo de distintos días.

Recordemos que en nuestros ejemplos anteriores, extraíamos los datos para un periodo de una semana (20210101-20210107) y necesitabamos 3 iteraciones (consultas) para extraer todas las paginas del informe completo. Quiere decir que al día podremos extraer los datos para 16-17 semanas aproximadamente (unos 4 meses aprox.).

Para facilitar el orden de las consultas lanzaremos nuestra celda para periodos de un mes, 4 veces al día, monitorizando el número de iteraciones que se hayan realizado y comprobando que todos los datos se han descargado correctamente. 

Por otra parte, recordemos que teníamos que descargar varios dataframes con distintas dimensiones para luego unirlos. Empezaremos por descargar el que anteriormente llamabamos first request xq tratarse de dimensiones con valores numericos que nos permitiran lanzar un primer MVP rápidamente de nuestro modelo de ML, mientras se descargan el resto de datos. 

Por último, sustituiremos el campo productName por productSKU que corresponde al número de referencia del producto. Esto nos evitará problemas en el caso de que en el transcurso de los 3 años se haya modificado el nombre del producto. Más adelante, cruzaremos ambas referencias con un dataset que relaciona el productSKU con el nombre del producto. 

#### DAY 1:
- DF: df_pDepth_Scount_daysSLS_srceMed
- PERIODS: 
    - January 2021
    - December 2020
    - November 2020
    - October 2020
- DIMENSIONS:
    - ga:productSKU
    - ga:pagePath
    - ga:dateHourMinute
    - ga:pageDepth
    - ga:sessionCount
    - ga:daysSinceLastSession
    - ga:sourceMedium
- METRICS:
    - ga:productListViews
    - ga:productListClicks
    - ga:productDetailViews
    - ga:productAddsToCart

In [95]:
total_requests_day1=0

In [96]:
# JANUARY 2021:

pagetokens = [None] + [str(x) for x in range(0,2100000,100000)][1:]
list_ = [] # empty list for row data
sampled_data = [] #empty list for sampling results
next_page_token_list = [] #empty list for next_page_tokens in each iteration
full_data_loaded = "All the data were loaded for this period and request" #Control message

for pagetoken in pagetokens:

    #REPORT REQUEST:
    response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'2021-01-01','endDate':'2021-01-31'}],
        'metrics':
        [
            {'expression':'ga:productListViews'},
            {'expression':'ga:productListClicks'},
            {'expression':'ga:productDetailViews'},
            {'expression':'ga:productAddsToCart'},
        ],
        'dimensions':
        [
            {'name':'ga:productSKU'},
            {'name':'ga:pagePath'},
            {'name':'ga:dateHourMinute'},
            {'name':'ga:pageDepth'},
            {'name':'ga:sessionCount'},
            {'name':'ga:daysSinceLastSession'},
            {'name':'ga:sourceMedium'},
        ],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        #'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        "pageToken": pagetoken, #extract data from pagetoken index
        'pageSize':100000 #if we do not specify we will get 1000 rows
    }]
}).execute()

    #PARSING THE REPORT DATA:
    
    for report in response.get('reports',[]):
        
        columnHeader = report.get('columnHeader',{})
        dimensionHeaders = columnHeader.get('dimensions',[])
        metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])
        
        rows = report.get('data',{}).get('rows',[])
        for row in rows:
            dict_={}
            dimensions = row.get('dimensions',[])
            dateRangeValues=row.get('metrics',[])
            
            for header, dimension in zip(dimensionHeaders,dimensions):
                dict_[header]=dimension
            
            for i,values in enumerate (dateRangeValues):
                for metricHeader,value in zip(metricHeaders,values.get('values')):
                    if ',' in value or '.' in value:
                      dict_[metricHeader.get('name')] = float(value)
                    else:
                      dict_[metricHeader.get('name')] = int(value)
                    
            list_.append(dict_)

        # Sampled Data? : 
        sampling = "No sampled data"
        sample_sizes = report.get('data',{}).get('samplesReadCounts',[])
        sample_spaces = report.get('data',{}).get('samplingSpaceSizes',[])
        if sample_sizes and sample_spaces:
            sampling = "Sampled data"
        sampled_data.append(sampling)
        
        # NextPageToken for following queries:    
        next_page_token = report.get('nextPageToken',None)
        next_page_token_list.append(str(next_page_token))
        
    if next_page_token == None: #to stop iterations once all the data are loaded
        break
    if int(next_page_token) > 1000000: #to warn about remaining pages
        full_data_loaded = "Remaining data to request. Please update pagetokens variable."

#DATAFRAME WITH DATA:

import pandas as pd
df_pDepth_Scount_daysSLS_srceMed_jan21= pd.DataFrame(list_)

iterations=len(next_page_token_list)
total_requests_day1 += iterations
remaining_requests_today = 50 - total_requests_day1

print((8*"*")+"Results of our request"+(8*"*"))
print("- Shape of the final dataframe: " + str(df_pDepth_Scount_daysSLS_srceMed_jan21.shape))
print("- Sampling for each iteration: " + str(sampled_data))
print("- Next page token for each iteration: " + str(next_page_token_list))
print("- Were all data exported? " + full_data_loaded)
print("- Number of requests or iterations: " + str(iterations))
print("- Remaining requests today: " + str(remaining_requests_today))
print("- Five first rows of our dataframe:")
df_pDepth_Scount_daysSLS_srceMed_jan21.head()

********Results of our request********
- Shape of the final dataframe: (983598, 11)
- Sampling for each iteration: ['No sampled data', 'No sampled data', 'No sampled data', 'No sampled data', 'No sampled data', 'No sampled data', 'No sampled data', 'No sampled data', 'No sampled data', 'No sampled data']
- Next page token for each iteration: ['100000', '200000', '300000', '400000', '500000', '600000', '700000', '800000', '900000', 'None']
- Were all data exported? All the data were loaded for this period and request
- Number of requests or iterations: 10
- Remaining requests today: 40
- Five first rows of our dataframe:


,ga:productSKU,ga:pagePath,ga:dateHourMinute,ga:pageDepth,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart
0,(other),(other),(other),(other),(other),(other),(other),53386,215,3050,265
1,1000,/es/821-dermocosmetica?page=3,202101021409,11,2,3,google / organic,1,0,0,0
2,1000,/es/821-dermocosmetica?page=3,202101061131,0,2,0,google / organic,1,0,0,0
3,1000,/es/821-dermocosmetica?page=3,202101082222,32,3,0,(direct) / (none),1,0,0,0
4,1000,/es/821-dermocosmetica?page=3,202101090041,7,12,1,google / organic,1,0,0,0


In [97]:
df_pDepth_Scount_daysSLS_srceMed_jan21.to_csv('../data/df_pDepth_Scount_daysSLS_srceMed_jan21.csv')

In [98]:
!ls ../data/

df_pDepth_Scount_daysSLS_srceMed_jan21.csv  first_df_january21.csv


In [99]:
# DECEMBER 2020:

pagetokens = [None] + [str(x) for x in range(0,2100000,100000)][1:]
list_ = [] # empty list for row data
sampled_data = [] #empty list for sampling results
next_page_token_list = [] #empty list for next_page_tokens in each iteration
full_data_loaded = "All the data were loaded for this period and request" #Control message

for pagetoken in pagetokens:

    #REPORT REQUEST:
    response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'2020-12-01','endDate':'2020-12-31'}],
        'metrics':
        [
            {'expression':'ga:productListViews'},
            {'expression':'ga:productListClicks'},
            {'expression':'ga:productDetailViews'},
            {'expression':'ga:productAddsToCart'},
        ],
        'dimensions':
        [
            {'name':'ga:productSKU'},
            {'name':'ga:pagePath'},
            {'name':'ga:dateHourMinute'},
            {'name':'ga:pageDepth'},
            {'name':'ga:sessionCount'},
            {'name':'ga:daysSinceLastSession'},
            {'name':'ga:sourceMedium'},
        ],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        #'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        "pageToken": pagetoken, #extract data from pagetoken index
        'pageSize':100000 #if we do not specify we will get 1000 rows
    }]
}).execute()

    #PARSING THE REPORT DATA:
    
    for report in response.get('reports',[]):
        
        columnHeader = report.get('columnHeader',{})
        dimensionHeaders = columnHeader.get('dimensions',[])
        metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])
        
        rows = report.get('data',{}).get('rows',[])
        for row in rows:
            dict_={}
            dimensions = row.get('dimensions',[])
            dateRangeValues=row.get('metrics',[])
            
            for header, dimension in zip(dimensionHeaders,dimensions):
                dict_[header]=dimension
            
            for i,values in enumerate (dateRangeValues):
                for metricHeader,value in zip(metricHeaders,values.get('values')):
                    if ',' in value or '.' in value:
                      dict_[metricHeader.get('name')] = float(value)
                    else:
                      dict_[metricHeader.get('name')] = int(value)
                    
            list_.append(dict_)

        # Sampled Data? : 
        sampling = "No sampled data"
        sample_sizes = report.get('data',{}).get('samplesReadCounts',[])
        sample_spaces = report.get('data',{}).get('samplingSpaceSizes',[])
        if sample_sizes and sample_spaces:
            sampling = "Sampled data"
        sampled_data.append(sampling)
        
        # NextPageToken for following queries:    
        next_page_token = report.get('nextPageToken',None)
        next_page_token_list.append(str(next_page_token))
        
    if next_page_token == None: #to stop iterations once all the data are loaded
        break
    if int(next_page_token) > 1000000: #to warn about remaining pages
        full_data_loaded = "Remaining data to request. Please update pagetokens variable."

#DATAFRAME WITH DATA:

import pandas as pd

df_pDepth_Scount_daysSLS_srceMed_dec20= pd.DataFrame(list_)

iterations=len(next_page_token_list)
total_requests_day1 += iterations
remaining_requests_today = 50 - total_requests_day1

print((8*"*")+"Results of our request"+(8*"*"))
print("- Shape of the final dataframe: " + str(df_pDepth_Scount_daysSLS_srceMed_dec20.shape))
print("- Sampling for each iteration: " + str(sampled_data))
print("- Next page token for each iteration: " + str(next_page_token_list))
print("- Were all data exported? " + full_data_loaded)
print("- Number of requests or iterations: " + str(iterations))
print("- Remaining requests today: " + str(remaining_requests_today))
print("- Five first rows of our dataframe:")
df_pDepth_Scount_daysSLS_srceMed_dec20.head()

********Results of our request********
- Shape of the final dataframe: (471866, 11)
- Sampling for each iteration: ['No sampled data', 'No sampled data', 'No sampled data', 'No sampled data', 'No sampled data']
- Next page token for each iteration: ['100000', '200000', '300000', '400000', 'None']
- Were all data exported? All the data were loaded for this period and request
- Number of requests or iterations: 5
- Remaining requests today: 35
- Five first rows of our dataframe:


,ga:productSKU,ga:pagePath,ga:dateHourMinute,ga:pageDepth,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart
0,1000,/es/821-dermocosmetica,202012092158,16,4,0,(direct) / (none),1,0,0,0
1,1000,/es/821-dermocosmetica?page=12,202012141651,8,2,0,google / organic,1,0,0,0
2,1000,/es/821-dermocosmetica?page=3,202012010932,6,3,0,google / organic,1,0,0,0
3,1000,/es/821-dermocosmetica?page=3,202012012307,21,3,6,google / organic,1,0,0,0
4,1000,/es/821-dermocosmetica?page=3,202012031849,13,1,0,google / organic,1,0,0,0


In [100]:
df_pDepth_Scount_daysSLS_srceMed_dec20.to_csv('../data/df_pDepth_Scount_daysSLS_srceMed_dec20.csv')

In [101]:
!ls ../data/

df_pDepth_Scount_daysSLS_srceMed_dec20.csv  first_df_january21.csv
df_pDepth_Scount_daysSLS_srceMed_jan21.csv


In [102]:
# NOVEMBER 2020:

pagetokens = [None] + [str(x) for x in range(0,2100000,100000)][1:]
list_ = [] # empty list for row data
sampled_data = [] #empty list for sampling results
next_page_token_list = [] #empty list for next_page_tokens in each iteration
full_data_loaded = "All the data were loaded for this period and request" #Control message

for pagetoken in pagetokens:

    #REPORT REQUEST:
    response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'2020-11-01','endDate':'2020-11-30'}],
        'metrics':
        [
            {'expression':'ga:productListViews'},
            {'expression':'ga:productListClicks'},
            {'expression':'ga:productDetailViews'},
            {'expression':'ga:productAddsToCart'},
        ],
        'dimensions':
        [
            {'name':'ga:productSKU'},
            {'name':'ga:pagePath'},
            {'name':'ga:dateHourMinute'},
            {'name':'ga:pageDepth'},
            {'name':'ga:sessionCount'},
            {'name':'ga:daysSinceLastSession'},
            {'name':'ga:sourceMedium'},
        ],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        #'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        "pageToken": pagetoken, #extract data from pagetoken index
        'pageSize':100000 #if we do not specify we will get 1000 rows
    }]
}).execute()

    #PARSING THE REPORT DATA:
    
    for report in response.get('reports',[]):
        
        columnHeader = report.get('columnHeader',{})
        dimensionHeaders = columnHeader.get('dimensions',[])
        metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])
        
        rows = report.get('data',{}).get('rows',[])
        for row in rows:
            dict_={}
            dimensions = row.get('dimensions',[])
            dateRangeValues=row.get('metrics',[])
            
            for header, dimension in zip(dimensionHeaders,dimensions):
                dict_[header]=dimension
            
            for i,values in enumerate (dateRangeValues):
                for metricHeader,value in zip(metricHeaders,values.get('values')):
                    if ',' in value or '.' in value:
                      dict_[metricHeader.get('name')] = float(value)
                    else:
                      dict_[metricHeader.get('name')] = int(value)
                    
            list_.append(dict_)

        # Sampled Data? : 
        sampling = "No sampled data"
        sample_sizes = report.get('data',{}).get('samplesReadCounts',[])
        sample_spaces = report.get('data',{}).get('samplingSpaceSizes',[])
        if sample_sizes and sample_spaces:
            sampling = "Sampled data"
        sampled_data.append(sampling)
        
        # NextPageToken for following queries:    
        next_page_token = report.get('nextPageToken',None)
        next_page_token_list.append(str(next_page_token))
        
    if next_page_token == None: #to stop iterations once all the data are loaded
        break
    if int(next_page_token) > 1000000: #to warn about remaining pages
        full_data_loaded = "Remaining data to request. Please update pagetokens variable."

#DATAFRAME WITH DATA:

import pandas as pd

df_pDepth_Scount_daysSLS_srceMed_nov20= pd.DataFrame(list_)

iterations=len(next_page_token_list)
total_requests_day1 += iterations
remaining_requests_today = 50 - total_requests_day1

print((8*"*")+"Results of our request"+(8*"*"))
print("- Shape of the final dataframe: " + str(df_pDepth_Scount_daysSLS_srceMed_nov20.shape))
print("- Sampling for each iteration: " + str(sampled_data))
print("- Next page token for each iteration: " + str(next_page_token_list))
print("- Were all data exported? " + full_data_loaded)
print("- Number of requests or iterations: " + str(iterations))
print("- Remaining requests today: " + str(remaining_requests_today))
print("- Five first rows of our dataframe:")
df_pDepth_Scount_daysSLS_srceMed_nov20.head()

********Results of our request********
- Shape of the final dataframe: (928954, 11)
- Sampling for each iteration: ['No sampled data', 'No sampled data', 'No sampled data', 'No sampled data', 'No sampled data', 'No sampled data', 'No sampled data', 'No sampled data', 'No sampled data', 'No sampled data']
- Next page token for each iteration: ['100000', '200000', '300000', '400000', '500000', '600000', '700000', '800000', '900000', 'None']
- Were all data exported? All the data were loaded for this period and request
- Number of requests or iterations: 10
- Remaining requests today: 25
- Five first rows of our dataframe:


,ga:productSKU,ga:pagePath,ga:dateHourMinute,ga:pageDepth,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart
0,1000,/es/821-dermocosmetica,202011281019,3,1,0,instagram.com / referral,1,0,0,0
1,1000,/es/821-dermocosmetica,202011281057,13,1,0,(direct) / (none),1,0,0,0
2,1000,/es/821-dermocosmetica?order=product.position....,202011181407,2,4,28,sendinblue / email,1,0,0,0
3,1000,/es/821-dermocosmetica?order=product.position....,202011210028,12,1,0,sendinblue / email,1,0,0,0
4,1000,/es/821-dermocosmetica?order=product.position....,202011252249,7,1,0,google / organic,1,0,0,0


In [103]:
df_pDepth_Scount_daysSLS_srceMed_nov20.to_csv('../data/df_pDepth_Scount_daysSLS_srceMed_nov20.csv')

In [104]:
!ls ../data/

df_pDepth_Scount_daysSLS_srceMed_dec20.csv
df_pDepth_Scount_daysSLS_srceMed_jan21.csv
df_pDepth_Scount_daysSLS_srceMed_nov20.csv
first_df_january21.csv


In [105]:
# OCTOBER 2020:

pagetokens = [None] + [str(x) for x in range(0,2100000,100000)][1:]
list_ = [] # empty list for row data
sampled_data = [] #empty list for sampling results
next_page_token_list = [] #empty list for next_page_tokens in each iteration
full_data_loaded = "All the data were loaded for this period and request" #Control message

for pagetoken in pagetokens:

    #REPORT REQUEST:
    response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'2020-10-01','endDate':'2020-10-31'}],
        'metrics':
        [
            {'expression':'ga:productListViews'},
            {'expression':'ga:productListClicks'},
            {'expression':'ga:productDetailViews'},
            {'expression':'ga:productAddsToCart'},
        ],
        'dimensions':
        [
            {'name':'ga:productSKU'},
            {'name':'ga:pagePath'},
            {'name':'ga:dateHourMinute'},
            {'name':'ga:pageDepth'},
            {'name':'ga:sessionCount'},
            {'name':'ga:daysSinceLastSession'},
            {'name':'ga:sourceMedium'},
        ],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        #'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        "pageToken": pagetoken, #extract data from pagetoken index
        'pageSize':100000 #if we do not specify we will get 1000 rows
    }]
}).execute()

    #PARSING THE REPORT DATA:
    
    for report in response.get('reports',[]):
        
        columnHeader = report.get('columnHeader',{})
        dimensionHeaders = columnHeader.get('dimensions',[])
        metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])
        
        rows = report.get('data',{}).get('rows',[])
        for row in rows:
            dict_={}
            dimensions = row.get('dimensions',[])
            dateRangeValues=row.get('metrics',[])
            
            for header, dimension in zip(dimensionHeaders,dimensions):
                dict_[header]=dimension
            
            for i,values in enumerate (dateRangeValues):
                for metricHeader,value in zip(metricHeaders,values.get('values')):
                    if ',' in value or '.' in value:
                      dict_[metricHeader.get('name')] = float(value)
                    else:
                      dict_[metricHeader.get('name')] = int(value)
                    
            list_.append(dict_)

        # Sampled Data? : 
        sampling = "No sampled data"
        sample_sizes = report.get('data',{}).get('samplesReadCounts',[])
        sample_spaces = report.get('data',{}).get('samplingSpaceSizes',[])
        if sample_sizes and sample_spaces:
            sampling = "Sampled data"
        sampled_data.append(sampling)
        
        # NextPageToken for following queries:    
        next_page_token = report.get('nextPageToken',None)
        next_page_token_list.append(str(next_page_token))
        
    if next_page_token == None: #to stop iterations once all the data are loaded
        break
    if int(next_page_token) > 1000000: #to warn about remaining pages
        full_data_loaded = "Remaining data to request. Please update pagetokens variable."

#DATAFRAME WITH DATA:

import pandas as pd

df_pDepth_Scount_daysSLS_srceMed_oct20= pd.DataFrame(list_)

iterations=len(next_page_token_list)
total_requests_day1 += iterations
remaining_requests_today = 50 - total_requests_day1

print((8*"*")+"Results of our request"+(8*"*"))
print("- Shape of the final dataframe: " + str(df_pDepth_Scount_daysSLS_srceMed_oct20.shape))
print("- Sampling for each iteration: " + str(sampled_data))
print("- Next page token for each iteration: " + str(next_page_token_list))
print("- Were all data exported? " + full_data_loaded)
print("- Number of requests or iterations: " + str(iterations))
print("- Remaining requests today: " + str(remaining_requests_today))
print("- Five first rows of our dataframe:")
df_pDepth_Scount_daysSLS_srceMed_oct20.head()

********Results of our request********
- Shape of the final dataframe: (135252, 11)
- Sampling for each iteration: ['No sampled data', 'No sampled data']
- Next page token for each iteration: ['100000', 'None']
- Were all data exported? All the data were loaded for this period and request
- Number of requests or iterations: 2
- Remaining requests today: 23
- Five first rows of our dataframe:


,ga:productSKU,ga:pagePath,ga:dateHourMinute,ga:pageDepth,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart
0,1000,/es/821-dermocosmetica?page=3,202010261903,12,52,0,webanalitica.es / referral,1,0,0,0
1,1000,/es/821-dermocosmetica?page=3,202010261904,21,1,0,(direct) / (none),1,0,0,0
2,1000,/es/821-dermocosmetica?page=3,202010270952,28,1,0,(direct) / (none),1,0,0,0
3,1000,/galileo/es/821-dermocosmetica,202010232132,17,1,0,(direct) / (none),1,0,0,0
4,1000,/galileo/es/821-dermocosmetica,202010240043,10,2,0,(direct) / (none),1,0,0,0


In [106]:
df_pDepth_Scount_daysSLS_srceMed_oct20.to_csv('../data/df_pDepth_Scount_daysSLS_srceMed_oct20.csv')

In [107]:
!ls ../data/

df_pDepth_Scount_daysSLS_srceMed_dec20.csv
df_pDepth_Scount_daysSLS_srceMed_jan21.csv
df_pDepth_Scount_daysSLS_srceMed_nov20.csv
df_pDepth_Scount_daysSLS_srceMed_oct20.csv
first_df_january21.csv


In [108]:
# SEPTEMBER 2020:

pagetokens = [None] + [str(x) for x in range(0,2100000,100000)][1:]
list_ = [] # empty list for row data
sampled_data = [] #empty list for sampling results
next_page_token_list = [] #empty list for next_page_tokens in each iteration
full_data_loaded = "All the data were loaded for this period and request" #Control message

for pagetoken in pagetokens:

    #REPORT REQUEST:
    response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'2020-09-01','endDate':'2020-09-30'}],
        'metrics':
        [
            {'expression':'ga:productListViews'},
            {'expression':'ga:productListClicks'},
            {'expression':'ga:productDetailViews'},
            {'expression':'ga:productAddsToCart'},
        ],
        'dimensions':
        [
            {'name':'ga:productSKU'},
            {'name':'ga:pagePath'},
            {'name':'ga:dateHourMinute'},
            {'name':'ga:pageDepth'},
            {'name':'ga:sessionCount'},
            {'name':'ga:daysSinceLastSession'},
            {'name':'ga:sourceMedium'},
        ],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        #'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        "pageToken": pagetoken, #extract data from pagetoken index
        'pageSize':100000 #if we do not specify we will get 1000 rows
    }]
}).execute()

    #PARSING THE REPORT DATA:
    
    for report in response.get('reports',[]):
        
        columnHeader = report.get('columnHeader',{})
        dimensionHeaders = columnHeader.get('dimensions',[])
        metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])
        
        rows = report.get('data',{}).get('rows',[])
        for row in rows:
            dict_={}
            dimensions = row.get('dimensions',[])
            dateRangeValues=row.get('metrics',[])
            
            for header, dimension in zip(dimensionHeaders,dimensions):
                dict_[header]=dimension
            
            for i,values in enumerate (dateRangeValues):
                for metricHeader,value in zip(metricHeaders,values.get('values')):
                    if ',' in value or '.' in value:
                      dict_[metricHeader.get('name')] = float(value)
                    else:
                      dict_[metricHeader.get('name')] = int(value)
                    
            list_.append(dict_)

        # Sampled Data? : 
        sampling = "No sampled data"
        sample_sizes = report.get('data',{}).get('samplesReadCounts',[])
        sample_spaces = report.get('data',{}).get('samplingSpaceSizes',[])
        if sample_sizes and sample_spaces:
            sampling = "Sampled data"
        sampled_data.append(sampling)
        
        # NextPageToken for following queries:    
        next_page_token = report.get('nextPageToken',None)
        next_page_token_list.append(str(next_page_token))
        
    if next_page_token == None: #to stop iterations once all the data are loaded
        break
    if int(next_page_token) > 1000000: #to warn about remaining pages
        full_data_loaded = "Remaining data to request. Please update pagetokens variable."

#DATAFRAME WITH DATA:

import pandas as pd

df_pDepth_Scount_daysSLS_srceMed_sep20= pd.DataFrame(list_)

iterations=len(next_page_token_list)
total_requests_day1 += iterations
remaining_requests_today = 50 - total_requests_day1

print((8*"*")+"Results of our request"+(8*"*"))
print("- Shape of the final dataframe: " + str(df_pDepth_Scount_daysSLS_srceMed_sep20.shape))
print("- Sampling for each iteration: " + str(sampled_data))
print("- Next page token for each iteration: " + str(next_page_token_list))
print("- Were all data exported? " + full_data_loaded)
print("- Number of requests or iterations: " + str(iterations))
print("- Remaining requests today: " + str(remaining_requests_today))
print("- Five first rows of our dataframe:")
df_pDepth_Scount_daysSLS_srceMed_sep20.head()

********Results of our request********
- Shape of the final dataframe: (158052, 11)
- Sampling for each iteration: ['No sampled data', 'No sampled data']
- Next page token for each iteration: ['100000', 'None']
- Were all data exported? All the data were loaded for this period and request
- Number of requests or iterations: 2
- Remaining requests today: 21
- Five first rows of our dataframe:


,ga:productSKU,ga:pagePath,ga:dateHourMinute,ga:pageDepth,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart
0,1000,/parafarmacia/es/134-☀-solares,202009021018,43,1,0,google / organic,2,0,0,0
1,1000,/parafarmacia/es/134-☀-solares,202009060100,7,1,0,google / organic,1,0,0,0
2,1000,/parafarmacia/es/134-☀-solares,202009091239,5,1,0,(direct) / (none),1,0,0,0
3,1000,/parafarmacia/es/134-☀-solares,202009091743,14,1,0,(direct) / (none),1,0,0,0
4,1000,/parafarmacia/es/134-☀-solares,202009102013,2,1,0,(direct) / (none),1,0,0,0


In [109]:
df_pDepth_Scount_daysSLS_srceMed_sep20.to_csv('../data/df_pDepth_Scount_daysSLS_srceMed_sep20.csv')

In [110]:
!ls ../data/

df_pDepth_Scount_daysSLS_srceMed_dec20.csv
df_pDepth_Scount_daysSLS_srceMed_jan21.csv
df_pDepth_Scount_daysSLS_srceMed_nov20.csv
df_pDepth_Scount_daysSLS_srceMed_oct20.csv
df_pDepth_Scount_daysSLS_srceMed_sep20.csv
first_df_january21.csv


In [111]:
# AUGUST 2020:

pagetokens = [None] + [str(x) for x in range(0,2100000,100000)][1:]
list_ = [] # empty list for row data
sampled_data = [] #empty list for sampling results
next_page_token_list = [] #empty list for next_page_tokens in each iteration
full_data_loaded = "All the data were loaded for this period and request" #Control message

for pagetoken in pagetokens:

    #REPORT REQUEST:
    response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'2020-08-01','endDate':'2020-08-31'}],
        'metrics':
        [
            {'expression':'ga:productListViews'},
            {'expression':'ga:productListClicks'},
            {'expression':'ga:productDetailViews'},
            {'expression':'ga:productAddsToCart'},
        ],
        'dimensions':
        [
            {'name':'ga:productSKU'},
            {'name':'ga:pagePath'},
            {'name':'ga:dateHourMinute'},
            {'name':'ga:pageDepth'},
            {'name':'ga:sessionCount'},
            {'name':'ga:daysSinceLastSession'},
            {'name':'ga:sourceMedium'},
        ],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        #'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        "pageToken": pagetoken, #extract data from pagetoken index
        'pageSize':100000 #if we do not specify we will get 1000 rows
    }]
}).execute()

    #PARSING THE REPORT DATA:
    
    for report in response.get('reports',[]):
        
        columnHeader = report.get('columnHeader',{})
        dimensionHeaders = columnHeader.get('dimensions',[])
        metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])
        
        rows = report.get('data',{}).get('rows',[])
        for row in rows:
            dict_={}
            dimensions = row.get('dimensions',[])
            dateRangeValues=row.get('metrics',[])
            
            for header, dimension in zip(dimensionHeaders,dimensions):
                dict_[header]=dimension
            
            for i,values in enumerate (dateRangeValues):
                for metricHeader,value in zip(metricHeaders,values.get('values')):
                    if ',' in value or '.' in value:
                      dict_[metricHeader.get('name')] = float(value)
                    else:
                      dict_[metricHeader.get('name')] = int(value)
                    
            list_.append(dict_)

        # Sampled Data? : 
        sampling = "No sampled data"
        sample_sizes = report.get('data',{}).get('samplesReadCounts',[])
        sample_spaces = report.get('data',{}).get('samplingSpaceSizes',[])
        if sample_sizes and sample_spaces:
            sampling = "Sampled data"
        sampled_data.append(sampling)
        
        # NextPageToken for following queries:    
        next_page_token = report.get('nextPageToken',None)
        next_page_token_list.append(str(next_page_token))
        
    if next_page_token == None: #to stop iterations once all the data are loaded
        break
    if int(next_page_token) > 1000000: #to warn about remaining pages
        full_data_loaded = "Remaining data to request. Please update pagetokens variable."

#DATAFRAME WITH DATA:

import pandas as pd

df_pDepth_Scount_daysSLS_srceMed_aug20= pd.DataFrame(list_)

iterations=len(next_page_token_list)
total_requests_day1 += iterations
remaining_requests_today = 50 - total_requests_day1

print((8*"*")+"Results of our request"+(8*"*"))
print("- Shape of the final dataframe: " + str(df_pDepth_Scount_daysSLS_srceMed_aug20.shape))
print("- Sampling for each iteration: " + str(sampled_data))
print("- Next page token for each iteration: " + str(next_page_token_list))
print("- Were all data exported? " + full_data_loaded)
print("- Number of requests or iterations: " + str(iterations))
print("- Remaining requests today: " + str(remaining_requests_today))
print("- Five first rows of our dataframe:")
df_pDepth_Scount_daysSLS_srceMed_aug20.head()

********Results of our request********
- Shape of the final dataframe: (117607, 11)
- Sampling for each iteration: ['No sampled data', 'No sampled data']
- Next page token for each iteration: ['100000', 'None']
- Were all data exported? All the data were loaded for this period and request
- Number of requests or iterations: 2
- Remaining requests today: 19
- Five first rows of our dataframe:


,ga:productSKU,ga:pagePath,ga:dateHourMinute,ga:pageDepth,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart
0,1000,/parafarmacia/es/1000-heliocare-360-pediatrics...,202008011924,1,1,0,google / cpc,0,0,1,0
1,1000,/parafarmacia/es/1000-heliocare-360-pediatrics...,202008012053,1,1,0,google / cpc,0,0,1,0
2,1000,/parafarmacia/es/1000-heliocare-360-pediatrics...,202008021128,1,1,0,google / cpc,0,0,1,0
3,1000,/parafarmacia/es/1000-heliocare-360-pediatrics...,202008031102,1,1,0,google / cpc,0,0,1,0
4,1000,/parafarmacia/es/1000-heliocare-360-pediatrics...,202008031455,1,1,0,google / cpc,0,0,1,0


In [112]:
df_pDepth_Scount_daysSLS_srceMed_aug20.to_csv('../data/df_pDepth_Scount_daysSLS_srceMed_aug20.csv')

In [113]:
!ls ../data/

df_pDepth_Scount_daysSLS_srceMed_aug20.csv
df_pDepth_Scount_daysSLS_srceMed_dec20.csv
df_pDepth_Scount_daysSLS_srceMed_jan21.csv
df_pDepth_Scount_daysSLS_srceMed_nov20.csv
df_pDepth_Scount_daysSLS_srceMed_oct20.csv
df_pDepth_Scount_daysSLS_srceMed_sep20.csv
first_df_january21.csv


In [114]:
# JULY 2020:

pagetokens = [None] + [str(x) for x in range(0,2100000,100000)][1:]
list_ = [] # empty list for row data
sampled_data = [] #empty list for sampling results
next_page_token_list = [] #empty list for next_page_tokens in each iteration
full_data_loaded = "All the data were loaded for this period and request" #Control message

for pagetoken in pagetokens:

    #REPORT REQUEST:
    response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'2020-07-01','endDate':'2020-07-31'}],
        'metrics':
        [
            {'expression':'ga:productListViews'},
            {'expression':'ga:productListClicks'},
            {'expression':'ga:productDetailViews'},
            {'expression':'ga:productAddsToCart'},
        ],
        'dimensions':
        [
            {'name':'ga:productSKU'},
            {'name':'ga:pagePath'},
            {'name':'ga:dateHourMinute'},
            {'name':'ga:pageDepth'},
            {'name':'ga:sessionCount'},
            {'name':'ga:daysSinceLastSession'},
            {'name':'ga:sourceMedium'},
        ],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        #'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        "pageToken": pagetoken, #extract data from pagetoken index
        'pageSize':100000 #if we do not specify we will get 1000 rows
    }]
}).execute()

    #PARSING THE REPORT DATA:
    
    for report in response.get('reports',[]):
        
        columnHeader = report.get('columnHeader',{})
        dimensionHeaders = columnHeader.get('dimensions',[])
        metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])
        
        rows = report.get('data',{}).get('rows',[])
        for row in rows:
            dict_={}
            dimensions = row.get('dimensions',[])
            dateRangeValues=row.get('metrics',[])
            
            for header, dimension in zip(dimensionHeaders,dimensions):
                dict_[header]=dimension
            
            for i,values in enumerate (dateRangeValues):
                for metricHeader,value in zip(metricHeaders,values.get('values')):
                    if ',' in value or '.' in value:
                      dict_[metricHeader.get('name')] = float(value)
                    else:
                      dict_[metricHeader.get('name')] = int(value)
                    
            list_.append(dict_)

        # Sampled Data? : 
        sampling = "No sampled data"
        sample_sizes = report.get('data',{}).get('samplesReadCounts',[])
        sample_spaces = report.get('data',{}).get('samplingSpaceSizes',[])
        if sample_sizes and sample_spaces:
            sampling = "Sampled data"
        sampled_data.append(sampling)
        
        # NextPageToken for following queries:    
        next_page_token = report.get('nextPageToken',None)
        next_page_token_list.append(str(next_page_token))
        
    if next_page_token == None: #to stop iterations once all the data are loaded
        break
    if int(next_page_token) > 1000000: #to warn about remaining pages
        full_data_loaded = "Remaining data to request. Please update pagetokens variable."

#DATAFRAME WITH DATA:

import pandas as pd

df_pDepth_Scount_daysSLS_srceMed_jul20= pd.DataFrame(list_)

iterations=len(next_page_token_list)
total_requests_day1 += iterations
remaining_requests_today = 50 - total_requests_day1

print((8*"*")+"Results of our request"+(8*"*"))
print("- Shape of the final dataframe: " + str(df_pDepth_Scount_daysSLS_srceMed_jul20.shape))
print("- Sampling for each iteration: " + str(sampled_data))
print("- Next page token for each iteration: " + str(next_page_token_list))
print("- Were all data exported? " + full_data_loaded)
print("- Number of requests or iterations: " + str(iterations))
print("- Remaining requests today: " + str(remaining_requests_today))
print("- Five first rows of our dataframe:")
df_pDepth_Scount_daysSLS_srceMed_jul20.head()

********Results of our request********
- Shape of the final dataframe: (124772, 11)
- Sampling for each iteration: ['No sampled data', 'No sampled data']
- Next page token for each iteration: ['100000', 'None']
- Were all data exported? All the data were loaded for this period and request
- Number of requests or iterations: 2
- Remaining requests today: 17
- Five first rows of our dataframe:


,ga:productSKU,ga:pagePath,ga:dateHourMinute,ga:pageDepth,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart
0,1000,/parafarmacia/es/1000-heliocare-360-pediatrics...,202007032325,5,1,0,google / organic,0,1,1,0
1,1000,/parafarmacia/es/1000-heliocare-360-pediatrics...,202007051430,29,1,0,(direct) / (none),0,1,1,0
2,1000,/parafarmacia/es/1000-heliocare-360-pediatrics...,202007051553,4,14,1,google / organic,0,1,1,0
3,1000,/parafarmacia/es/1000-heliocare-360-pediatrics...,202007051903,3,1,0,google / organic,0,1,1,0
4,1000,/parafarmacia/es/1000-heliocare-360-pediatrics...,202007052245,1,15,0,google / organic,0,1,1,0


In [115]:
df_pDepth_Scount_daysSLS_srceMed_jul20.to_csv('../data/df_pDepth_Scount_daysSLS_srceMed_jul20.csv')

In [116]:
!ls ../data/

df_pDepth_Scount_daysSLS_srceMed_aug20.csv
df_pDepth_Scount_daysSLS_srceMed_dec20.csv
df_pDepth_Scount_daysSLS_srceMed_jan21.csv
df_pDepth_Scount_daysSLS_srceMed_jul20.csv
df_pDepth_Scount_daysSLS_srceMed_nov20.csv
df_pDepth_Scount_daysSLS_srceMed_oct20.csv
df_pDepth_Scount_daysSLS_srceMed_sep20.csv
first_df_january21.csv


In [117]:
# JUNE 2020:

pagetokens = [None] + [str(x) for x in range(0,2100000,100000)][1:]
list_ = [] # empty list for row data
sampled_data = [] #empty list for sampling results
next_page_token_list = [] #empty list for next_page_tokens in each iteration
full_data_loaded = "All the data were loaded for this period and request" #Control message

for pagetoken in pagetokens:

    #REPORT REQUEST:
    response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'2020-06-01','endDate':'2020-06-30'}],
        'metrics':
        [
            {'expression':'ga:productListViews'},
            {'expression':'ga:productListClicks'},
            {'expression':'ga:productDetailViews'},
            {'expression':'ga:productAddsToCart'},
        ],
        'dimensions':
        [
            {'name':'ga:productSKU'},
            {'name':'ga:pagePath'},
            {'name':'ga:dateHourMinute'},
            {'name':'ga:pageDepth'},
            {'name':'ga:sessionCount'},
            {'name':'ga:daysSinceLastSession'},
            {'name':'ga:sourceMedium'},
        ],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        #'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        "pageToken": pagetoken, #extract data from pagetoken index
        'pageSize':100000 #if we do not specify we will get 1000 rows
    }]
}).execute()

    #PARSING THE REPORT DATA:
    
    for report in response.get('reports',[]):
        
        columnHeader = report.get('columnHeader',{})
        dimensionHeaders = columnHeader.get('dimensions',[])
        metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])
        
        rows = report.get('data',{}).get('rows',[])
        for row in rows:
            dict_={}
            dimensions = row.get('dimensions',[])
            dateRangeValues=row.get('metrics',[])
            
            for header, dimension in zip(dimensionHeaders,dimensions):
                dict_[header]=dimension
            
            for i,values in enumerate (dateRangeValues):
                for metricHeader,value in zip(metricHeaders,values.get('values')):
                    if ',' in value or '.' in value:
                      dict_[metricHeader.get('name')] = float(value)
                    else:
                      dict_[metricHeader.get('name')] = int(value)
                    
            list_.append(dict_)

        # Sampled Data? : 
        sampling = "No sampled data"
        sample_sizes = report.get('data',{}).get('samplesReadCounts',[])
        sample_spaces = report.get('data',{}).get('samplingSpaceSizes',[])
        if sample_sizes and sample_spaces:
            sampling = "Sampled data"
        sampled_data.append(sampling)
        
        # NextPageToken for following queries:    
        next_page_token = report.get('nextPageToken',None)
        next_page_token_list.append(str(next_page_token))
        
    if next_page_token == None: #to stop iterations once all the data are loaded
        break
    if int(next_page_token) > 1000000: #to warn about remaining pages
        full_data_loaded = "Remaining data to request. Please update pagetokens variable."

#DATAFRAME WITH DATA:

import pandas as pd

df_pDepth_Scount_daysSLS_srceMed_jun20= pd.DataFrame(list_)

iterations=len(next_page_token_list)
total_requests_day1 += iterations
remaining_requests_today = 50 - total_requests_day1

print((8*"*")+"Results of our request"+(8*"*"))
print("- Shape of the final dataframe: " + str(df_pDepth_Scount_daysSLS_srceMed_jun20.shape))
print("- Sampling for each iteration: " + str(sampled_data))
print("- Next page token for each iteration: " + str(next_page_token_list))
print("- Were all data exported? " + full_data_loaded)
print("- Number of requests or iterations: " + str(iterations))
print("- Remaining requests today: " + str(remaining_requests_today))
print("- Five first rows of our dataframe:")
df_pDepth_Scount_daysSLS_srceMed_jun20.head()

********Results of our request********
- Shape of the final dataframe: (153456, 11)
- Sampling for each iteration: ['No sampled data', 'No sampled data']
- Next page token for each iteration: ['100000', 'None']
- Were all data exported? All the data were loaded for this period and request
- Number of requests or iterations: 2
- Remaining requests today: 15
- Five first rows of our dataframe:


,ga:productSKU,ga:pagePath,ga:dateHourMinute,ga:pageDepth,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart
0,1000,/parafarmacia/es/,202006141541,5,3,13,google / organic,0,0,0,1
1,1000,/parafarmacia/es/1000-heliocare-360-pediatrics...,202006061027,4,1,0,google / organic,0,1,1,0
2,1000,/parafarmacia/es/1000-heliocare-360-pediatrics...,202006071444,21,6,10,google / organic,0,1,1,0
3,1000,/parafarmacia/es/1000-heliocare-360-pediatrics...,202006110731,7,1,0,google / organic,0,1,1,0
4,1000,/parafarmacia/es/1000-heliocare-360-pediatrics...,202006111002,5,1,0,google / organic,0,1,1,0


In [118]:
df_pDepth_Scount_daysSLS_srceMed_jun20.to_csv('../data/df_pDepth_Scount_daysSLS_srceMed_jun20.csv')

In [119]:
!ls ../data/

df_pDepth_Scount_daysSLS_srceMed_aug20.csv
df_pDepth_Scount_daysSLS_srceMed_dec20.csv
df_pDepth_Scount_daysSLS_srceMed_jan21.csv
df_pDepth_Scount_daysSLS_srceMed_jul20.csv
df_pDepth_Scount_daysSLS_srceMed_jun20.csv
df_pDepth_Scount_daysSLS_srceMed_nov20.csv
df_pDepth_Scount_daysSLS_srceMed_oct20.csv
df_pDepth_Scount_daysSLS_srceMed_sep20.csv
first_df_january21.csv


In [120]:
# MAY 2020:

pagetokens = [None] + [str(x) for x in range(0,2100000,100000)][1:]
list_ = [] # empty list for row data
sampled_data = [] #empty list for sampling results
next_page_token_list = [] #empty list for next_page_tokens in each iteration
full_data_loaded = "All the data were loaded for this period and request" #Control message

for pagetoken in pagetokens:

    #REPORT REQUEST:
    response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'2020-05-01','endDate':'2020-05-31'}],
        'metrics':
        [
            {'expression':'ga:productListViews'},
            {'expression':'ga:productListClicks'},
            {'expression':'ga:productDetailViews'},
            {'expression':'ga:productAddsToCart'},
        ],
        'dimensions':
        [
            {'name':'ga:productSKU'},
            {'name':'ga:pagePath'},
            {'name':'ga:dateHourMinute'},
            {'name':'ga:pageDepth'},
            {'name':'ga:sessionCount'},
            {'name':'ga:daysSinceLastSession'},
            {'name':'ga:sourceMedium'},
        ],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        #'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        "pageToken": pagetoken, #extract data from pagetoken index
        'pageSize':100000 #if we do not specify we will get 1000 rows
    }]
}).execute()

    #PARSING THE REPORT DATA:
    
    for report in response.get('reports',[]):
        
        columnHeader = report.get('columnHeader',{})
        dimensionHeaders = columnHeader.get('dimensions',[])
        metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])
        
        rows = report.get('data',{}).get('rows',[])
        for row in rows:
            dict_={}
            dimensions = row.get('dimensions',[])
            dateRangeValues=row.get('metrics',[])
            
            for header, dimension in zip(dimensionHeaders,dimensions):
                dict_[header]=dimension
            
            for i,values in enumerate (dateRangeValues):
                for metricHeader,value in zip(metricHeaders,values.get('values')):
                    if ',' in value or '.' in value:
                      dict_[metricHeader.get('name')] = float(value)
                    else:
                      dict_[metricHeader.get('name')] = int(value)
                    
            list_.append(dict_)

        # Sampled Data? : 
        sampling = "No sampled data"
        sample_sizes = report.get('data',{}).get('samplesReadCounts',[])
        sample_spaces = report.get('data',{}).get('samplingSpaceSizes',[])
        if sample_sizes and sample_spaces:
            sampling = "Sampled data"
        sampled_data.append(sampling)
        
        # NextPageToken for following queries:    
        next_page_token = report.get('nextPageToken',None)
        next_page_token_list.append(str(next_page_token))
        
    if next_page_token == None: #to stop iterations once all the data are loaded
        break
    if int(next_page_token) > 1000000: #to warn about remaining pages
        full_data_loaded = "Remaining data to request. Please update pagetokens variable."

#DATAFRAME WITH DATA:

import pandas as pd

df_pDepth_Scount_daysSLS_srceMed_may20= pd.DataFrame(list_)

iterations=len(next_page_token_list)
total_requests_day1 += iterations
remaining_requests_today = 50 - total_requests_day1

print((8*"*")+"Results of our request"+(8*"*"))
print("- Shape of the final dataframe: " + str(df_pDepth_Scount_daysSLS_srceMed_may20.shape))
print("- Sampling for each iteration: " + str(sampled_data))
print("- Next page token for each iteration: " + str(next_page_token_list))
print("- Were all data exported? " + full_data_loaded)
print("- Number of requests or iterations: " + str(iterations))
print("- Remaining requests today: " + str(remaining_requests_today))
print("- Five first rows of our dataframe:")
df_pDepth_Scount_daysSLS_srceMed_may20.head()

********Results of our request********
- Shape of the final dataframe: (317238, 11)
- Sampling for each iteration: ['No sampled data', 'No sampled data', 'No sampled data', 'No sampled data']
- Next page token for each iteration: ['100000', '200000', '300000', 'None']
- Were all data exported? All the data were loaded for this period and request
- Number of requests or iterations: 4
- Remaining requests today: 11
- Five first rows of our dataframe:


,ga:productSKU,ga:pagePath,ga:dateHourMinute,ga:pageDepth,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart
0,1000,/parafarmacia/es/1000-heliocare-360-pediatrics...,202005011603,14,4,8,instagram.com / referral,0,1,1,0
1,1000,/parafarmacia/es/1000-heliocare-360-pediatrics...,202005021522,8,1,0,(direct) / (none),0,1,1,0
2,1000,/parafarmacia/es/1000-heliocare-360-pediatrics...,202005051052,27,1,0,google / cpc,0,1,1,0
3,1000,/parafarmacia/es/1000-heliocare-360-pediatrics...,202005051210,25,3,0,google / organic,0,1,1,0
4,1000,/parafarmacia/es/1000-heliocare-360-pediatrics...,202005051214,25,3,0,google / organic,0,1,1,0


In [121]:
df_pDepth_Scount_daysSLS_srceMed_may20.to_csv('../data/df_pDepth_Scount_daysSLS_srceMed_may20.csv')

In [122]:
!ls ../data/

df_pDepth_Scount_daysSLS_srceMed_aug20.csv
df_pDepth_Scount_daysSLS_srceMed_dec20.csv
df_pDepth_Scount_daysSLS_srceMed_jan21.csv
df_pDepth_Scount_daysSLS_srceMed_jul20.csv
df_pDepth_Scount_daysSLS_srceMed_jun20.csv
df_pDepth_Scount_daysSLS_srceMed_may20.csv
df_pDepth_Scount_daysSLS_srceMed_nov20.csv
df_pDepth_Scount_daysSLS_srceMed_oct20.csv
df_pDepth_Scount_daysSLS_srceMed_sep20.csv
first_df_january21.csv


In [123]:
# APRIL 2020:

pagetokens = [None] + [str(x) for x in range(0,2100000,100000)][1:]
list_ = [] # empty list for row data
sampled_data = [] #empty list for sampling results
next_page_token_list = [] #empty list for next_page_tokens in each iteration
full_data_loaded = "All the data were loaded for this period and request" #Control message

for pagetoken in pagetokens:

    #REPORT REQUEST:
    response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'2020-04-01','endDate':'2020-04-30'}],
        'metrics':
        [
            {'expression':'ga:productListViews'},
            {'expression':'ga:productListClicks'},
            {'expression':'ga:productDetailViews'},
            {'expression':'ga:productAddsToCart'},
        ],
        'dimensions':
        [
            {'name':'ga:productSKU'},
            {'name':'ga:pagePath'},
            {'name':'ga:dateHourMinute'},
            {'name':'ga:pageDepth'},
            {'name':'ga:sessionCount'},
            {'name':'ga:daysSinceLastSession'},
            {'name':'ga:sourceMedium'},
        ],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        #'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        "pageToken": pagetoken, #extract data from pagetoken index
        'pageSize':100000 #if we do not specify we will get 1000 rows
    }]
}).execute()

    #PARSING THE REPORT DATA:
    
    for report in response.get('reports',[]):
        
        columnHeader = report.get('columnHeader',{})
        dimensionHeaders = columnHeader.get('dimensions',[])
        metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])
        
        rows = report.get('data',{}).get('rows',[])
        for row in rows:
            dict_={}
            dimensions = row.get('dimensions',[])
            dateRangeValues=row.get('metrics',[])
            
            for header, dimension in zip(dimensionHeaders,dimensions):
                dict_[header]=dimension
            
            for i,values in enumerate (dateRangeValues):
                for metricHeader,value in zip(metricHeaders,values.get('values')):
                    if ',' in value or '.' in value:
                      dict_[metricHeader.get('name')] = float(value)
                    else:
                      dict_[metricHeader.get('name')] = int(value)
                    
            list_.append(dict_)

        # Sampled Data? : 
        sampling = "No sampled data"
        sample_sizes = report.get('data',{}).get('samplesReadCounts',[])
        sample_spaces = report.get('data',{}).get('samplingSpaceSizes',[])
        if sample_sizes and sample_spaces:
            sampling = "Sampled data"
        sampled_data.append(sampling)
        
        # NextPageToken for following queries:    
        next_page_token = report.get('nextPageToken',None)
        next_page_token_list.append(str(next_page_token))
        
    if next_page_token == None: #to stop iterations once all the data are loaded
        break
    if int(next_page_token) > 1000000: #to warn about remaining pages
        full_data_loaded = "Remaining data to request. Please update pagetokens variable."

#DATAFRAME WITH DATA:

import pandas as pd

df_pDepth_Scount_daysSLS_srceMed_apr20= pd.DataFrame(list_)

iterations=len(next_page_token_list)
total_requests_day1 += iterations
remaining_requests_today = 50 - total_requests_day1

print((8*"*")+"Results of our request"+(8*"*"))
print("- Shape of the final dataframe: " + str(df_pDepth_Scount_daysSLS_srceMed_apr20.shape))
print("- Sampling for each iteration: " + str(sampled_data))
print("- Next page token for each iteration: " + str(next_page_token_list))
print("- Were all data exported? " + full_data_loaded)
print("- Number of requests or iterations: " + str(iterations))
print("- Remaining requests today: " + str(remaining_requests_today))
print("- Five first rows of our dataframe:")
df_pDepth_Scount_daysSLS_srceMed_apr20.head()

********Results of our request********
- Shape of the final dataframe: (332187, 11)
- Sampling for each iteration: ['No sampled data', 'No sampled data', 'No sampled data', 'No sampled data']
- Next page token for each iteration: ['100000', '200000', '300000', 'None']
- Were all data exported? All the data were loaded for this period and request
- Number of requests or iterations: 4
- Remaining requests today: 7
- Five first rows of our dataframe:


,ga:productSKU,ga:pagePath,ga:dateHourMinute,ga:pageDepth,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart
0,1000,/parafarmacia/es/1000-heliocare-360-pediatrics...,202004051552,10,2,5,(direct) / (none),0,1,1,0
1,1000,/parafarmacia/es/1000-heliocare-360-pediatrics...,202004151902,16,15,0,l.instagram.com / referral,0,1,1,0
2,1000,/parafarmacia/es/1000-heliocare-360-pediatrics...,202004271119,11,2,1,google / organic,0,1,1,0
3,1000,/parafarmacia/es/1000-heliocare-360-pediatrics...,202004291133,9,3,2,l.instagram.com / referral,0,1,1,0
4,1000,/parafarmacia/es/285-protectores-solares-infantil,202004150242,42,2,0,google / organic,1,0,0,0


In [124]:
df_pDepth_Scount_daysSLS_srceMed_apr20.to_csv('../data/df_pDepth_Scount_daysSLS_srceMed_apr20.csv')

In [125]:
!ls ../data/

df_pDepth_Scount_daysSLS_srceMed_apr20.csv
df_pDepth_Scount_daysSLS_srceMed_aug20.csv
df_pDepth_Scount_daysSLS_srceMed_dec20.csv
df_pDepth_Scount_daysSLS_srceMed_jan21.csv
df_pDepth_Scount_daysSLS_srceMed_jul20.csv
df_pDepth_Scount_daysSLS_srceMed_jun20.csv
df_pDepth_Scount_daysSLS_srceMed_may20.csv
df_pDepth_Scount_daysSLS_srceMed_nov20.csv
df_pDepth_Scount_daysSLS_srceMed_oct20.csv
df_pDepth_Scount_daysSLS_srceMed_sep20.csv
first_df_january21.csv


In [126]:
# MARCH 2020:

pagetokens = [None] + [str(x) for x in range(0,2100000,100000)][1:]
list_ = [] # empty list for row data
sampled_data = [] #empty list for sampling results
next_page_token_list = [] #empty list for next_page_tokens in each iteration
full_data_loaded = "All the data were loaded for this period and request" #Control message

for pagetoken in pagetokens:

    #REPORT REQUEST:
    response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'2020-03-01','endDate':'2020-03-31'}],
        'metrics':
        [
            {'expression':'ga:productListViews'},
            {'expression':'ga:productListClicks'},
            {'expression':'ga:productDetailViews'},
            {'expression':'ga:productAddsToCart'},
        ],
        'dimensions':
        [
            {'name':'ga:productSKU'},
            {'name':'ga:pagePath'},
            {'name':'ga:dateHourMinute'},
            {'name':'ga:pageDepth'},
            {'name':'ga:sessionCount'},
            {'name':'ga:daysSinceLastSession'},
            {'name':'ga:sourceMedium'},
        ],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        #'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        "pageToken": pagetoken, #extract data from pagetoken index
        'pageSize':100000 #if we do not specify we will get 1000 rows
    }]
}).execute()

    #PARSING THE REPORT DATA:
    
    for report in response.get('reports',[]):
        
        columnHeader = report.get('columnHeader',{})
        dimensionHeaders = columnHeader.get('dimensions',[])
        metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])
        
        rows = report.get('data',{}).get('rows',[])
        for row in rows:
            dict_={}
            dimensions = row.get('dimensions',[])
            dateRangeValues=row.get('metrics',[])
            
            for header, dimension in zip(dimensionHeaders,dimensions):
                dict_[header]=dimension
            
            for i,values in enumerate (dateRangeValues):
                for metricHeader,value in zip(metricHeaders,values.get('values')):
                    if ',' in value or '.' in value:
                      dict_[metricHeader.get('name')] = float(value)
                    else:
                      dict_[metricHeader.get('name')] = int(value)
                    
            list_.append(dict_)

        # Sampled Data? : 
        sampling = "No sampled data"
        sample_sizes = report.get('data',{}).get('samplesReadCounts',[])
        sample_spaces = report.get('data',{}).get('samplingSpaceSizes',[])
        if sample_sizes and sample_spaces:
            sampling = "Sampled data"
        sampled_data.append(sampling)
        
        # NextPageToken for following queries:    
        next_page_token = report.get('nextPageToken',None)
        next_page_token_list.append(str(next_page_token))
        
    if next_page_token == None: #to stop iterations once all the data are loaded
        break
    if int(next_page_token) > 1000000: #to warn about remaining pages
        full_data_loaded = "Remaining data to request. Please update pagetokens variable."

#DATAFRAME WITH DATA:

import pandas as pd

df_pDepth_Scount_daysSLS_srceMed_mar20= pd.DataFrame(list_)

iterations=len(next_page_token_list)
total_requests_day1 += iterations
remaining_requests_today = 50 - total_requests_day1

print((8*"*")+"Results of our request"+(8*"*"))
print("- Shape of the final dataframe: " + str(df_pDepth_Scount_daysSLS_srceMed_mar20.shape))
print("- Sampling for each iteration: " + str(sampled_data))
print("- Next page token for each iteration: " + str(next_page_token_list))
print("- Were all data exported? " + full_data_loaded)
print("- Number of requests or iterations: " + str(iterations))
print("- Remaining requests today: " + str(remaining_requests_today))
print("- Five first rows of our dataframe:")
df_pDepth_Scount_daysSLS_srceMed_mar20.head()

********Results of our request********
- Shape of the final dataframe: (92916, 11)
- Sampling for each iteration: ['No sampled data']
- Next page token for each iteration: ['None']
- Were all data exported? All the data were loaded for this period and request
- Number of requests or iterations: 1
- Remaining requests today: 6
- Five first rows of our dataframe:


,ga:productSKU,ga:pagePath,ga:dateHourMinute,ga:pageDepth,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart
0,1000,/parafarmacia/es/busqueda?orderby=position&ord...,202003261422,6,1,0,google / organic,1,0,0,0
1,1000,/parafarmacia/es/busqueda?orderby=position&ord...,202003311706,2,2,79,google / organic,1,0,0,0
2,1000,/parafarmacia/es/busqueda?orderby=position&ord...,202003290030,1,3,0,google / organic,1,0,0,0
3,1000,/parafarmacia/es/busqueda?orderby=position&ord...,202003182152,24,1,0,(direct) / (none),1,0,0,0
4,1000,/parafarmacia/es/busqueda?orderby=position&ord...,202003251636,2,1,0,(direct) / (none),1,0,0,0


In [127]:
df_pDepth_Scount_daysSLS_srceMed_mar20.to_csv('../data/df_pDepth_Scount_daysSLS_srceMed_mar20.csv')

In [128]:
!ls ../data/

df_pDepth_Scount_daysSLS_srceMed_apr20.csv
df_pDepth_Scount_daysSLS_srceMed_aug20.csv
df_pDepth_Scount_daysSLS_srceMed_dec20.csv
df_pDepth_Scount_daysSLS_srceMed_jan21.csv
df_pDepth_Scount_daysSLS_srceMed_jul20.csv
df_pDepth_Scount_daysSLS_srceMed_jun20.csv
df_pDepth_Scount_daysSLS_srceMed_mar20.csv
df_pDepth_Scount_daysSLS_srceMed_may20.csv
df_pDepth_Scount_daysSLS_srceMed_nov20.csv
df_pDepth_Scount_daysSLS_srceMed_oct20.csv
df_pDepth_Scount_daysSLS_srceMed_sep20.csv
first_df_january21.csv


In [129]:
# FEBRUARY 2020:

pagetokens = [None] + [str(x) for x in range(0,2100000,100000)][1:]
list_ = [] # empty list for row data
sampled_data = [] #empty list for sampling results
next_page_token_list = [] #empty list for next_page_tokens in each iteration
full_data_loaded = "All the data were loaded for this period and request" #Control message

for pagetoken in pagetokens:

    #REPORT REQUEST:
    response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'2020-02-01','endDate':'2020-02-29'}],
        'metrics':
        [
            {'expression':'ga:productListViews'},
            {'expression':'ga:productListClicks'},
            {'expression':'ga:productDetailViews'},
            {'expression':'ga:productAddsToCart'},
        ],
        'dimensions':
        [
            {'name':'ga:productSKU'},
            {'name':'ga:pagePath'},
            {'name':'ga:dateHourMinute'},
            {'name':'ga:pageDepth'},
            {'name':'ga:sessionCount'},
            {'name':'ga:daysSinceLastSession'},
            {'name':'ga:sourceMedium'},
        ],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        #'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        "pageToken": pagetoken, #extract data from pagetoken index
        'pageSize':100000 #if we do not specify we will get 1000 rows
    }]
}).execute()

    #PARSING THE REPORT DATA:
    
    for report in response.get('reports',[]):
        
        columnHeader = report.get('columnHeader',{})
        dimensionHeaders = columnHeader.get('dimensions',[])
        metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])
        
        rows = report.get('data',{}).get('rows',[])
        for row in rows:
            dict_={}
            dimensions = row.get('dimensions',[])
            dateRangeValues=row.get('metrics',[])
            
            for header, dimension in zip(dimensionHeaders,dimensions):
                dict_[header]=dimension
            
            for i,values in enumerate (dateRangeValues):
                for metricHeader,value in zip(metricHeaders,values.get('values')):
                    if ',' in value or '.' in value:
                      dict_[metricHeader.get('name')] = float(value)
                    else:
                      dict_[metricHeader.get('name')] = int(value)
                    
            list_.append(dict_)

        # Sampled Data? : 
        sampling = "No sampled data"
        sample_sizes = report.get('data',{}).get('samplesReadCounts',[])
        sample_spaces = report.get('data',{}).get('samplingSpaceSizes',[])
        if sample_sizes and sample_spaces:
            sampling = "Sampled data"
        sampled_data.append(sampling)
        
        # NextPageToken for following queries:    
        next_page_token = report.get('nextPageToken',None)
        next_page_token_list.append(str(next_page_token))
        
    if next_page_token == None: #to stop iterations once all the data are loaded
        break
    if int(next_page_token) > 1000000: #to warn about remaining pages
        full_data_loaded = "Remaining data to request. Please update pagetokens variable."

#DATAFRAME WITH DATA:

import pandas as pd

df_pDepth_Scount_daysSLS_srceMed_feb20= pd.DataFrame(list_)

iterations=len(next_page_token_list)
total_requests_day1 += iterations
remaining_requests_today = 50 - total_requests_day1

print((8*"*")+"Results of our request"+(8*"*"))
print("- Shape of the final dataframe: " + str(df_pDepth_Scount_daysSLS_srceMed_feb20.shape))
print("- Sampling for each iteration: " + str(sampled_data))
print("- Next page token for each iteration: " + str(next_page_token_list))
print("- Were all data exported? " + full_data_loaded)
print("- Number of requests or iterations: " + str(iterations))
print("- Remaining requests today: " + str(remaining_requests_today))
print("- Five first rows of our dataframe:")
df_pDepth_Scount_daysSLS_srceMed_feb20.head()

********Results of our request********
- Shape of the final dataframe: (28417, 11)
- Sampling for each iteration: ['No sampled data']
- Next page token for each iteration: ['None']
- Were all data exported? All the data were loaded for this period and request
- Number of requests or iterations: 1
- Remaining requests today: 5
- Five first rows of our dataframe:


,ga:productSKU,ga:pagePath,ga:dateHourMinute,ga:pageDepth,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart
0,1000,/parafarmacia/es/1000-heliocare-360-pediatrics...,202002162211,13,6,7,instagram.com / referral,0,1,1,0
1,1001,/parafarmacia/es/1001-heliocare-caps-60-caps.html,202002111440,1,1,0,google / cpc,0,0,1,0
2,1001,/parafarmacia/es/1001-heliocare-caps-60-caps.html,202002122318,1,1,0,google / cpc,0,0,1,0
3,1001,/parafarmacia/es/1001-heliocare-caps-60-caps.html,202002160929,10,5,3,(direct) / (none),0,1,1,0
4,1001,/parafarmacia/es/1001-heliocare-caps-60-caps.html,202002251214,1,1,0,google / cpc,0,0,1,0


In [130]:
df_pDepth_Scount_daysSLS_srceMed_feb20.to_csv('../data/df_pDepth_Scount_daysSLS_srceMed_feb20.csv')

In [131]:
!ls ../data/

df_pDepth_Scount_daysSLS_srceMed_apr20.csv
df_pDepth_Scount_daysSLS_srceMed_aug20.csv
df_pDepth_Scount_daysSLS_srceMed_dec20.csv
df_pDepth_Scount_daysSLS_srceMed_feb20.csv
df_pDepth_Scount_daysSLS_srceMed_jan21.csv
df_pDepth_Scount_daysSLS_srceMed_jul20.csv
df_pDepth_Scount_daysSLS_srceMed_jun20.csv
df_pDepth_Scount_daysSLS_srceMed_mar20.csv
df_pDepth_Scount_daysSLS_srceMed_may20.csv
df_pDepth_Scount_daysSLS_srceMed_nov20.csv
df_pDepth_Scount_daysSLS_srceMed_oct20.csv
df_pDepth_Scount_daysSLS_srceMed_sep20.csv
first_df_january21.csv


In [132]:
# JANUARY 2020:

pagetokens = [None] + [str(x) for x in range(0,2100000,100000)][1:]
list_ = [] # empty list for row data
sampled_data = [] #empty list for sampling results
next_page_token_list = [] #empty list for next_page_tokens in each iteration
full_data_loaded = "All the data were loaded for this period and request" #Control message

for pagetoken in pagetokens:

    #REPORT REQUEST:
    response = service.reports().batchGet(
body={
    'reportRequests':[{
        'viewId':'151304036',
        'dateRanges':[{'startDate':'2020-01-01','endDate':'2020-01-31'}],
        'metrics':
        [
            {'expression':'ga:productListViews'},
            {'expression':'ga:productListClicks'},
            {'expression':'ga:productDetailViews'},
            {'expression':'ga:productAddsToCart'},
        ],
        'dimensions':
        [
            {'name':'ga:productSKU'},
            {'name':'ga:pagePath'},
            {'name':'ga:dateHourMinute'},
            {'name':'ga:pageDepth'},
            {'name':'ga:sessionCount'},
            {'name':'ga:daysSinceLastSession'},
            {'name':'ga:sourceMedium'},
        ],
        #"filtersExpression":"ga:pagePath=~products;ga:pagePath!@/translate",
        #'orderBys':[{"fieldName":"ga:sessions","sortOrder":"DESCENDING"}],
        "pageToken": pagetoken, #extract data from pagetoken index
        'pageSize':100000 #if we do not specify we will get 1000 rows
    }]
}).execute()

    #PARSING THE REPORT DATA:
    
    for report in response.get('reports',[]):
        
        columnHeader = report.get('columnHeader',{})
        dimensionHeaders = columnHeader.get('dimensions',[])
        metricHeaders = columnHeader.get('metricHeader',{}).get('metricHeaderEntries',[])
        
        rows = report.get('data',{}).get('rows',[])
        for row in rows:
            dict_={}
            dimensions = row.get('dimensions',[])
            dateRangeValues=row.get('metrics',[])
            
            for header, dimension in zip(dimensionHeaders,dimensions):
                dict_[header]=dimension
            
            for i,values in enumerate (dateRangeValues):
                for metricHeader,value in zip(metricHeaders,values.get('values')):
                    if ',' in value or '.' in value:
                      dict_[metricHeader.get('name')] = float(value)
                    else:
                      dict_[metricHeader.get('name')] = int(value)
                    
            list_.append(dict_)

        # Sampled Data? : 
        sampling = "No sampled data"
        sample_sizes = report.get('data',{}).get('samplesReadCounts',[])
        sample_spaces = report.get('data',{}).get('samplingSpaceSizes',[])
        if sample_sizes and sample_spaces:
            sampling = "Sampled data"
        sampled_data.append(sampling)
        
        # NextPageToken for following queries:    
        next_page_token = report.get('nextPageToken',None)
        next_page_token_list.append(str(next_page_token))
        
    if next_page_token == None: #to stop iterations once all the data are loaded
        break
    if int(next_page_token) > 1000000: #to warn about remaining pages
        full_data_loaded = "Remaining data to request. Please update pagetokens variable."

#DATAFRAME WITH DATA:

import pandas as pd

df_pDepth_Scount_daysSLS_srceMed_jan20= pd.DataFrame(list_)

iterations=len(next_page_token_list)
total_requests_day1 += iterations
remaining_requests_today = 50 - total_requests_day1

print((8*"*")+"Results of our request"+(8*"*"))
print("- Shape of the final dataframe: " + str(df_pDepth_Scount_daysSLS_srceMed_jan20.shape))
print("- Sampling for each iteration: " + str(sampled_data))
print("- Next page token for each iteration: " + str(next_page_token_list))
print("- Were all data exported? " + full_data_loaded)
print("- Number of requests or iterations: " + str(iterations))
print("- Remaining requests today: " + str(remaining_requests_today))
print("- Five first rows of our dataframe:")
df_pDepth_Scount_daysSLS_srceMed_jan20.head()

********Results of our request********
- Shape of the final dataframe: (24830, 11)
- Sampling for each iteration: ['No sampled data']
- Next page token for each iteration: ['None']
- Were all data exported? All the data were loaded for this period and request
- Number of requests or iterations: 1
- Remaining requests today: 4
- Five first rows of our dataframe:


,ga:productSKU,ga:pagePath,ga:dateHourMinute,ga:pageDepth,ga:sessionCount,ga:daysSinceLastSession,ga:sourceMedium,ga:productListViews,ga:productListClicks,ga:productDetailViews,ga:productAddsToCart
0,1000,/parafarmacia/es/1000-heliocare-360-pediatrics...,202001141125,12,407,0,google / organic,0,1,1,0
1,1000,/parafarmacia/es/1000-heliocare-360-pediatrics...,202001141345,5,3,0,(direct) / (none),0,0,1,0
2,1001,/parafarmacia/es/1001-heliocare-caps-60-caps.html,202001080932,6,3,19,google / organic,0,1,1,0
3,1002,/parafarmacia/es/147-champus,202001161511,14,1,0,google / cpc,2,0,0,0
4,1002,/parafarmacia/es/147-champus,202001162342,14,3,0,google / cpc,1,0,0,0


In [133]:
df_pDepth_Scount_daysSLS_srceMed_jan20.to_csv('../data/df_pDepth_Scount_daysSLS_srceMed_jan20.csv')

In [134]:
!ls ../data/

df_pDepth_Scount_daysSLS_srceMed_apr20.csv
df_pDepth_Scount_daysSLS_srceMed_aug20.csv
df_pDepth_Scount_daysSLS_srceMed_dec20.csv
df_pDepth_Scount_daysSLS_srceMed_feb20.csv
df_pDepth_Scount_daysSLS_srceMed_jan20.csv
df_pDepth_Scount_daysSLS_srceMed_jan21.csv
df_pDepth_Scount_daysSLS_srceMed_jul20.csv
df_pDepth_Scount_daysSLS_srceMed_jun20.csv
df_pDepth_Scount_daysSLS_srceMed_mar20.csv
df_pDepth_Scount_daysSLS_srceMed_may20.csv
df_pDepth_Scount_daysSLS_srceMed_nov20.csv
df_pDepth_Scount_daysSLS_srceMed_oct20.csv
df_pDepth_Scount_daysSLS_srceMed_sep20.csv
first_df_january21.csv
